# Notebook Predict Codes

## 📄 Introdução

&nbsp;&nbsp;&nbsp;&nbsp;Este notebook criado pelo grupo Predict Codes contém a análise de dados da Rede Gazeta. O objetivo é identificar padrões de receita, analisar o impacto de diferentes veículos, setores e segmentos para que, assim, seja possível prever a receita futura.
<br>
&nbsp;&nbsp;&nbsp;&nbsp; Agora, para a plena execução do notebook, é necessário primeiramente acessar [este link do drive](https://drive.google.com/drive/folders/1gPtxw2ZwFKJgWMN1R-kwkeerCG9_fNwz?usp=drive_link) com o email institucional do Inteli, baixar os arquivos necessários e inserir eles na pasta "base_de_dados", dentro da pasta "notebooks". Após isso, será possível executar o comando Run All.

## 📂 Carregamento de dados

Nessa etapa, serão instaladas e importadas as principais bibliotecas que serão utilizadas em todo o notebook, bem como a importação das bases de dados utilizadas no modelo preditivo.

In [ ]:
! pip install pandas matplotlib seaborn scikit-learn scipy lightgbm optuna ipywidgets pingouin

In [2]:
# Importação das principais bibliotecas
import itertools
import warnings

# Manipulação de dados e estatísticas
import numpy as np
import pandas as pd
from scipy import stats

# Plotagem de gráficos e visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Widgets interativos
import ipywidgets as widgets
from IPython.display import display

# Machine Learning
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error, 
    mean_absolute_percentage_error, 
    mean_squared_error, 
    r2_score
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Otimização
import optuna

# Configurações de exibição do Pandas e alertas
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")


In [3]:
# Importação das bases de dados
# Base de de dados de agosto (principal)
df = pd.read_csv('base_de_dados/BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.xlsx - BASE.csv')
# Base de de dados de ocup
df_ocup = pd.read_csv('base_de_dados/BASE INTELI_META_OCUP.xlsx - OCUPAÇÃO .csv')
# Base de dados de meta
df_meta = pd.read_csv('base_de_dados/BASE INTELI_META_OCUP.xlsx - META.csv')
# Base de dados de audiência
df_audiencia = pd.read_csv('base_de_dados/AUDIENCIA_07_08_2024_COMPLETA.xlsx - AUDIENCIA.csv')
# Base de dados econômicos
df_dados_economicos = pd.read_csv('base_de_dados/DadosEconomicos_ES_Inteli.xlsx - VarejoTotal.csv')



## 🔎 Exploração dos dados

&nbsp;&nbsp;&nbsp;&nbsp;A exploração de dados pode ser compreendida como a etapa inicial para o tratamento dos dados, na qual os se examinam suas principais características, padrões e distribuições.

### Identificação das colunas numéricas e categóricas

&nbsp;&nbsp;&nbsp;&nbsp;Ao realizar a exploração de dados, é necessário compreender quais são os tipos dos dados que compõem as bases de dados utilizadas. Para a realização da classificação das colunas, foi feita uma análise visual, na qual observamos e classificamos manualmente os dados presentes em cada coluna. Esse método foi necessário devido a inconsistências identificadas na descrição automática dos tipos de dados por meio do código, que apresentava erros, que podem ser obvervados abaixo. Esses erros foram corrigidos posteriormente durante a etapa de limpeza dos dados, permitindo uma classificação mais precisa entre colunas numéricas e categóricas.

#### Base de dados principal:

In [ ]:
# Exibe os tipos de dados de cada coluna no df principal
df.dtypes

#### Base de dados de meta/ocupação:

In [ ]:
# Exibe os tipos de dados de cada coluna no df_meta
df_meta.dtypes

In [ ]:
# Exibe os tipos de dados de cada coluna no df_ocup
df_ocup.dtypes

#### Base de dados de audiência:

In [ ]:
# Exibe os tipos de dados de cada coluna no df_audiencia
df_audiencia.dtypes

### Estatística descritiva das colunas

Base de dados principal:

In [ ]:
df.describe()

Base de dados econômicos:

In [ ]:
df_dados_economicos.describe()

Base de dados de audiência:

In [ ]:
df_audiencia.describe()

### Gráficos para visualização 

In [ ]:
# Código para implantação dos gráficos
corr_df = df.select_dtypes(include=[np.number])
corr_matrix = corr_df.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()

## 🛠️ Pré-Processamento dos dados

&nbsp;&nbsp;&nbsp;&nbsp;Etapa fundamental para evitar o surgimento de vieses nos dados. Dados que são mal preparados ou não passam por um bom pré-processamento podem diminuir a assertividade do modelo preditivo.
Ao longo dessa seção, as bases de dados foram limpadas, corrigidas e normalizadas.

### ⚠️ Tratamento de missing values, identificação de outliers e correção

&nbsp;&nbsp;&nbsp;&nbsp;Nessa etapa foram tratados os valores ausentes e duplicados, corrigido os tipos de colunas e identificado os outliers da base de dados principal, de meta/ocupação e de audiência.

#### Base de dados principal:

Ao longo desse bloco de códigos, foram tratados os problemas existentes nessa base de dados, como missing values, colunas numéricas que estão com tipo "object", identificação de outliers, remoção de colunas com valores ausentes.

In [ ]:
# Identificação dos valores nulos
df.isnull().sum()

In [ ]:
# Verificação das colunas numéricas que estão sendo tratadas como objetos
df.info()

In [14]:
# Remoção de coluna com valores vazios
df = df.dropna(axis=1, thresh=int(0.6 * len(df)))

In [ ]:
# Correção das colunas numéricas
colunas_numericas = ['VL Tabela', 'Desconto R$','Desc %', 'Vl Bruto', 'Vl Liquido Final', 'IPCA ES', 'IPCA BR', 'Taxa Ac. TRI % PIB']
for coluna in colunas_numericas:
  if df[coluna].dtype == 'object':
    df[coluna] = df[coluna].str.replace('.', '')
    df[coluna] = df[coluna].str.replace(',', '.')
    df[coluna] = df[coluna].str.replace('%', '.')
    df[coluna] = df[coluna].astype(float)
df.head()

In [16]:
# Os valores nulos da coluna do PIB dizem respeito ao segundo trimestre de 2024
# Como o resultado ainda não foi divulgado, os valores nulos foram completados com estimativas do resultado do PIB
df['Taxa Ac. TRI % PIB'].fillna(2.1, inplace = True)

In [ ]:
df = df.dropna(subset=['Setor'])
df

In [ ]:
# Verificação se ainda há valores faltantes
df.isnull().sum()

In [ ]:
colunas_principais = ['VL Tabela', 'Vl Bruto', 'Vl Liquido Final']
# Calcular o Z-Score para cada coluna numérica
z_scores_df = stats.zscore(df[colunas_principais])

# Criar um DataFrame com os Z-Scores
z_scores_df = pd.DataFrame(z_scores_df, columns=colunas_principais)

# Identificar os outliers (considerando Z-Score > 3 ou < -3 como outliers)
outliers_df = (z_scores_df > 3) | (z_scores_df < -3)

# Visualizar as linhas que possuem outliers em alguma das colunas
outliers_df = df[outliers_df.any(axis=1)]

outliers_df

In [ ]:
# Verificar o menor outlier em cada uma das colunas
menor_outlier_vl_tabela = outliers_df['VL Tabela'].min()
menor_outlier_vl_bruto = outliers_df['Vl Bruto'].min()
menor_outlier_vl_liquido_final = outliers_df['Vl Liquido Final'].min()

# Exibir os menores outliers para cada coluna
print(f"Menor outlier em 'VL Tabela': {menor_outlier_vl_tabela}")
print(f"Menor outlier em 'Vl Bruto': {menor_outlier_vl_bruto}")
print(f"Menor outlier em 'Vl Liquido Final': {menor_outlier_vl_liquido_final}")


In [ ]:
# Identificação dos valores duplicados
principal_duplicadas = df[df.duplicated()]
principal_duplicadas

In [22]:
# Remoção das linhas duplicadas
df.drop_duplicates(inplace=True)

#### Base de dados de meta/ocupação:

Na base de meta/ocupação, verificou-se que não existiam outliers, valores faltando e linhas duplicadas. No entanto, foi necessário a adequação da coluna de mês ano e ocupação(%) para valores mais adequados para se trabalhar.

In [ ]:
# Identificação dos valores nulos
df_ocup.isnull().sum()

In [ ]:
# Verificação das colunas numéricas que estão sendo tratadas como objetos
df_ocup.info()

In [ ]:
# Transformando as datas em um formato de datetime
meses = {
    'jan.': '01',
    'fev.': '02',
    'mar.': '03',
    'abr.': '04',
    'mai.': '05',
    'jun.': '06',
    'jul.': '07',
    'ago.': '08',
    'set.': '09',
    'out.': '10',
    'nov.': '11',
    'dez.': '12'
}

for mes_pt, mes_num in meses.items():
    df_ocup['Mês/ano'] = df_ocup['Mês/ano'].str.replace(mes_pt, mes_num)

df_ocup['Mês/ano'] = pd.to_datetime(df_ocup['Mês/ano'], format='%m-%y')

# Transformando a coluna de 'Ocupação (%)' em numérica
if df_ocup['Ocupação (%)'].dtype == 'object':
    df_ocup['Ocupação (%)'] = df_ocup['Ocupação (%)'].str.replace('%', '')
    df_ocup['Ocupação (%)'] = df_ocup['Ocupação (%)'].astype(float)
df_ocup.dtypes

In [ ]:
# Calcular o Z-Score para a coluna de Ocupação (%)
z_scores_df_ocup = stats.zscore(df_ocup['Ocupação (%)'])

# Criar um DataFrame com os Z-Scores
z_scores_df_ocup = pd.DataFrame(z_scores_df_ocup)

# Exibir os primeiros valores dos Z-Scores
z_scores_df_ocup

In [ ]:
# Identificar os outliers (considerando Z-Score > 3 ou < -3 como outliers)
outliers_df_ocup = (z_scores_df_ocup > 3) | (z_scores_df_ocup < -3)

# Visualizar as linhas que possuem outliers na coluna de Ocupação (%)
outliers_df_ocup = df_ocup[outliers_df_ocup.any(axis=1)]

outliers_df_ocup

In [ ]:
# Identificação dos valores duplicados
ocup_duplicadas = df_ocup[df_ocup.duplicated()]
ocup_duplicadas

#### Base de dados de audiência:

Na limpeza da base de dados de audiência, notou-se que não haviam valores duplicados e faltantes. Havia a presença apenas de um outlier, e foi necessário transformar as colunas de Internet e "Portal G1/GE/Home" de string para numéricas.

In [ ]:
# Identificação dos valores nulos
df_audiencia.isnull().sum()

In [ ]:
# Verificação das colunas numéricas que estão sendo tratadas como objetos
df_audiencia.info()

In [ ]:
# Transformando colunas em string para numéricas
colunas_audiencia = ['GAZETA FM VITÓRIA', 'LITORAL FM', 'CBN VITORIA', 
                     'RÁDIO MIX VITÓRIA', 'INTERNET', 'PORTAL G1/GE/HOME']
colunas_audiencia_string = ['INTERNET', 'PORTAL G1/GE/HOME']

for coluna in colunas_audiencia_string:
  if df_audiencia[coluna].dtype == 'object':
    df_audiencia[coluna] = df_audiencia[coluna].str.replace('.', '')
    df_audiencia[coluna] = df_audiencia[coluna].astype(float)
df_audiencia.dtypes

In [ ]:
# Calcular o Z-Score para cada coluna numérica
z_scores_df_audiencia = stats.zscore(df_audiencia[colunas_audiencia])

# Criar um DataFrame com os Z-Scores
z_scores_df_audiencia = pd.DataFrame(z_scores_df_audiencia, columns=colunas_audiencia)

# Identificar os outliers (considerando Z-Score > 3 ou < -3 como outliers)
outliers_df_audiencia = (z_scores_df_audiencia > 3) | (z_scores_df_audiencia < -3)

# Visualizar as linhas que possuem outliers em alguma das colunas
outliers_df_audiencia = df_audiencia[outliers_df_audiencia.any(axis=1)]

outliers_df_audiencia

In [ ]:
# Identificação dos valores duplicados
duplicadas_audiencia = df_audiencia[df_audiencia.duplicated()]
duplicadas_audiencia

In [ ]:
df.describe()

In [35]:
# cria coluna de Vl Liquido Final sem normalização para ser usada na modelo LightGBM
df['Vl Liquido Final s/ normalização'] = df['Vl Liquido Final']

In [36]:
df['Vl Liquido Final s/ norm'] = df['Vl Liquido Final']

In [ ]:
df_veiculo_grafico = df.groupby('Veiculo')['Vl Liquido Final'].sum().reset_index()
df_veiculo_grafico.sort_values(by='Vl Liquido Final', ascending=False, inplace=True)
df_veiculo_principais = df_veiculo_grafico.head(2)
df_veiculo_menores = df_veiculo_grafico.tail(17)
df_veiculo_menores['Vl Liquido Final'].sum()

grafico_pizza = [df_veiculo_menores['Vl Liquido Final'].sum(), df_veiculo_principais['Vl Liquido Final'].sum()]
keys = ['Outros veículos', 'TV Gazeta e Internet']
explode = (0.1, 0) 
# plotting data on chart 
plt.pie(grafico_pizza, labels=keys, explode=explode, shadow=True, startangle=90,
        autopct='%.0f%%') 
  
# displaying chart 
plt.show() 

### 📏 Normalização das variáveis

&nbsp;&nbsp;&nbsp;&nbsp;A normalização de variáveis numéricas é um processo realizado para a transformação dos dados para uma escala comum, feita para garantir que não haja uma dominância de algumas variáveis no modelo, devido sua magnitude. Portanto, realizou-se a análise da distribuição dos dados das colunas das nossas base de dados para assim compreender quais necessitavam de normalização e depois essa foi aplicada como é demonstrado abaixo.

#### Base de dados principal

&nbsp;&nbsp;&nbsp;&nbsp;A base de dados possui as seguintes colunas numéricas: 'VL Tabela', 'Desconto R$', 'Desc %', 'Vl Bruto', 'Vl Liquido Final', 'IPCA ES', 'IPCA BR' e 'Taxa Ac. TRI % PIB'. A análise descritiva revelou uma grande disparidade nos valores relacionados à receita e ao desconto, o que justifica a aplicação da transformação logarítmica para normalizar essas colunas e tratar os outliers presentes.

In [ ]:
# Análise descritiva das colunas de receita
df[['VL Tabela', 'Desconto R$', 'Vl Bruto', 'Vl Liquido Final']].describe()

&nbsp;&nbsp;&nbsp;&nbsp;Para aplicar essa normalização, utilizamos a função `np.log1p`, que calcula o logaritmo natural de (1 + valor) para cada dado. Sendo uma transformação, adequada para ajustar variáveis com ampla variação, pois reduz a diferença entre valores altos e baixos, estabilizando a variância. 

##### Coluna "VL Tabela"

&nbsp;&nbsp;&nbsp;&nbsp;Primeiramente, realizou-se a normalização, por meio da Transformação Logarítmica, da coluna "VL Tabela":

In [39]:
# Aplica a transformação logarítmica à coluna 'VL Tabela'
df['VL Tabela'] = np.log1p(df['VL Tabela'])

#Adiciona o método a uma variável para que ele possa ser utilizado na normalização
scaler = StandardScaler()

df['VL Tabela'] = scaler.fit_transform(df[['VL Tabela']])


&nbsp;&nbsp;&nbsp;&nbsp;Ao aplicar a transformação logarítmica, conseguimos reduzir a assimetria da distribuição dos dados na coluna 'VL Tabela'. Isso pode ser observado no gráfico de distribuição a seguir, onde a forma dos dados está mais centralizada e próxima de uma distribuição normal.

In [ ]:
cor_barras = "lightsalmon"
cor_kde = "black"

# Gráfico de Histograma
sns.histplot(df['VL Tabela'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")

# Adicionar o KDE separadamente com cor e label personalizados
sns.kdeplot(df['VL Tabela'], color=cor_kde, label='Curva de Densidade Estimada (KDE)')

# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right", fontsize=6)  # Adiciona a legenda no canto superior direito

# Ajustando o título e os eixos
plt.title('Gráfico de Distribuição: VL Tabela"', fontsize=12)
plt.xlabel('                                     Valores de VL Tabela               ln(1+x)', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

plt.show()

##### Coluna "Desconto R$" 

&nbsp;&nbsp;&nbsp;&nbsp;Antes de realizar a normalização, foi necessário a transformação dos valores de descontos negativos em valores absolutos, para que assim o método de normalização escolhido realizasse a normalização da forma esperada.

In [41]:
# Altera os valores da coluna de desconto para valores absolutos
df[['Desconto R$']] = df[['Desconto R$']].abs()

&nbsp;&nbsp;&nbsp;&nbsp;Após a adequação dos valores, aplicou-se o mesmo método de normalização da coluna anterior:

In [42]:
# Aplica a transformação logarítmica à coluna 'Desconto R$'
df['Desconto R$'] = np.log1p(df['Desconto R$'])
df['Desconto R$']= scaler.fit_transform(df[['Desconto R$']])


&nbsp;&nbsp;&nbsp;&nbsp;Assim, percebe-se que transformação logarítmica reduziu a assimetria na coluna 'Desconto R$', melhorando a distribuição dos dados. Embora ainda haja concentração em torno de zero, a curva de densidade mostra uma distribuição mais equilibrada, como é possível observar no gráfico abaixo.

In [ ]:
# Gráfico de Distribuição com a linha de densidade (KDE)
sns.histplot(df['Desconto R$'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")
sns.kdeplot(df['Desconto R$'], color=cor_kde, linewidth=1.3, label="Curva de Densidade Estimada (KDE)")  # Limita a KDE para começar no 0

# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right")  # Adiciona a legenda no canto superior direito

# Ajustando o título e os eixos
plt.title('Gráfico de Distribuição dos Valores da Coluna "Desconto R$"', fontsize=12)
plt.xlabel('                                                       Valores da coluna "Desconto R$"                                       ln(1+x)', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

# Remover legendas extras e linhas desnecessárias
plt.grid(False)
plt.tight_layout()

# Exibindo o gráfico
plt.show()

##### Coluna "Vl Bruto" 

In [44]:
# Aplica a transformação logarítmica à coluna "Vl Bruto"
df['Vl Bruto'] = np.log1p(df['Vl Bruto'])
df['Vl Bruto'] = scaler.fit_transform(df[['Vl Bruto']])


&nbsp;&nbsp;&nbsp;&nbsp;Assim como nas colunas acima, a aplicação da normalização reduziu a dispersão dos dados, aproximando os valores de uma distribuição normal, como é possível observar no gráfico abaixo.

In [ ]:
# Gráfico de Histograma
sns.histplot(df['Vl Bruto'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")

# Adicionar o KDE separadamente com cor e label personalizados
sns.kdeplot(df['Vl Bruto'], color=cor_kde, label='Curva de Densidade Estimada (KDE)')

# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right", fontsize=6)  # Adiciona a legenda no canto superior direito

# Ajustando o título e os eixos
plt.title('Gráfico de Distribuição: Vl Bruto"', fontsize=12)
plt.xlabel('                                                       Valores de Vl Bruto                                       ln(1+x)', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

plt.show()

##### Coluna "Vl Liquido Final"

In [ ]:
# Aplica a transformação logarítmica à coluna "Vl Liquido Final"
df_sem_escala = df.copy()
df['Vl Liquido Final'] = np.log1p(df['Vl Liquido Final'])
df['Vl Liquido Final'] = scaler.fit_transform(df[['Vl Liquido Final']])
df['Vl Liquido Final']


&nbsp;&nbsp;&nbsp;&nbsp;Por fim, a normalização da coluna "VL Líquido Final" também apresentou os resultados esperados, possibilitando uma redução na variabilidade dos dados e atenuando a influência dos outliers.

In [ ]:
# Gráfico de Histograma
sns.histplot(df['Vl Bruto'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")

# Adicionar o KDE separadamente com cor e label personalizados
sns.kdeplot(df['Vl Bruto'], color=cor_kde, label='Curva de Densidade Estimada (KDE)')

# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right", fontsize=6)  # Adiciona a legenda no canto superior direito

# Ajustando o título e os eixos
plt.title('Gráfico de Distribuição: Vl Liquido Final', fontsize=12)
plt.xlabel('                              Valores de Vl Liquido Final           ln(1+x)', fontsize=12)
plt.ylabel('Densidade', fontsize=12)

plt.show()

#### Base de dados econômicos


Na base de dados Economicos , existem três colunas com variáveis numéricas que necessitavam ser normalizadas para que pudessem ser utilizadas em modelos preditivos e análises estatísticas. As colunas normalizadas foram as  'PMC - Número-índice (2022=100) (Número-índice)' , 'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)'e 'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)'.


In [48]:
#Simplifica os nomes das colunas para que elas fiquem com uma visualização mais simplificada e direta
df_dados_economicos.rename(columns={'PMC - Número-índice (2022=100) (Número-índice)': 'pmc_indice',
                                    'PMC - Número-índice com ajuste sazonal (2022=100) (Número-índice)': 'pmc_indice_sazonal',
                                    'PMC - Variação mês/mês imediatamente anterior, com ajuste sazonal (M/M-1) (%)': 'pmc_variacao' }, inplace=True)


In [49]:
#Deleta duas linhas da tabela onde continham dados faltantes
df_dados_economicos = df_dados_economicos.drop(index=0)
df_dados_economicos = df_dados_economicos.drop(index=1)

In [ ]:
#Transforma as colunas as células strings em numéricas
col_num_economicos = ['pmc_indice', 'pmc_indice_sazonal', 'pmc_variacao']
for coluna in col_num_economicos:
  if df_dados_economicos[coluna].dtype == 'object':
    df_dados_economicos[coluna] = df_dados_economicos[coluna].str.replace(',', '.')
    df_dados_economicos[coluna] = df_dados_economicos[coluna].astype(float)
df_dados_economicos.head()


Escolher o método Z-Score de Normalização é ideal para dados que seguem uma distribuição aproximadamente normal (esse é o caso das colunas dessa tabela), pois ele transforma os dados para uma escala com média 0 e desvio padrão 1. Isso é especialmente útil para modelos estatísticos que assumem normalidade e são sensíveis à escala dos dados, como regressão linear e modelos baseados em distância, garantindo consistência e comparabilidade entre as variáveis.

In [ ]:
#Aplica o método "Z-Score Normalization" às colunas numéricas
df_dados_economicos[col_num_economicos] = scaler.fit_transform(df_dados_economicos[col_num_economicos])

df_dados_economicos.describe()


##### Coluna "pmc_indice"

Demonstração do Gráfico de Densidada da coluna "pmc_indice'"

In [ ]:
sns.histplot(df_dados_economicos['pmc_indice'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")
# Adicionar o KDE separadamente com cor e label personalizados
sns.kdeplot(df_dados_economicos['pmc_indice'], color=cor_kde, label='Curva de Densidade Estimada (KDE)')
plt.title('Gráfico de Distribuição: pmc_indice', fontsize=12)
# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right", fontsize=6)  # Adiciona a legenda no canto superior direito
plt.xlabel('Número-Índice do PMC')
plt.ylabel('Densidade')
plt.show()

##### Coluna "mpc_indice_sazonal"

Demonstração do Gráfico de Densidada da coluna "pmc_variacao"

In [ ]:
sns.histplot(df_dados_economicos['pmc_indice_sazonal'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")
# Adicionar o KDE separadamente com cor e label personalizados
sns.kdeplot(df_dados_economicos['pmc_indice_sazonal'], color=cor_kde, label='Curva de Densidade Estimada (KDE)')
plt.title('Gráfico de Distribuição: pmc_indice_sazonal', fontsize=12)
# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right", fontsize=6)  # Adiciona a legenda no canto superior direito
plt.xlabel('Índice Sazonal do PMC')
plt.ylabel('Densidade')
plt.show()

##### Coluna "pmc_variacao"

Demonstração do Gráfico de Densidada da coluna "pmc_variacao"

In [ ]:
sns.histplot(df_dados_economicos['pmc_variacao'], kde=False, stat="density", color=cor_barras, label="Histograma de Frequência")
# Adicionar o KDE separadamente com cor e label personalizados
sns.kdeplot(df_dados_economicos['pmc_variacao'], color=cor_kde, label='Curva de Densidade Estimada (KDE)')
plt.title('Gráfico de Distribuição: pmc_variacao', fontsize=12)
# Adicionando a legenda
plt.legend(title="Legenda", loc="upper right", fontsize=6)  # Adiciona a legenda no canto superior direito
plt.xlabel('Variação do PMC')
plt.ylabel('Densidade')
plt.show()

Agora é mostrado o começinho da tabela para que seja possível visualizar um pedaço do resultado

In [ ]:
df_dados_economicos.head()

###  🔤 Codificação das variáveis categóricas


A aplicação de variáveis categóricas em análise de dados e machine learning envolve a conversão de dados qualitativos, como rótulos ou categorias, em uma forma numérica que pode ser compreendida por algoritmos. Variáveis categóricas representam diferentes grupos ou níveis, como "Mês", "Setor" ou "Origem", e são utilizados para titular de forma qualitativa certos atributos em um conjunto de dados. A correta codificação dessas variáveis é indispensável, pois permite que modelos preditivos e análises estatísticas utilizem informações categóricas de maneira eficaz, garantindo a integridade dos resultados e a melhoria da precisão dos modelos. Também é importante ressaltar que a codificação errada de variáveis pode acarretar em grave erro de cálculo nos modelos preditivos.

Foi utilizado inicialmente o método de 'Label Encoder' para codificar todas as colunas que continham variáveis categóricas. 

In [56]:
# Adiciona à variável um método de Label Encoder, que irá aplicar o método Label às colunas
le = LabelEncoder()

#Faz com que o programa pare de realizar downcasting, pois isso será impossibilitado de acontecer \
# nas próximas versões do pandas e estava deixando uma código de exeção após algumas codificações
pd.set_option('future.no_silent_downcasting', True)

#### Base de dados principal:


Na base de dados principal, foram identificadas colunas com variáveis categóricas que necessitavam de codificação para que pudessem ser utilizadas em modelos preditivos e análises estatísticas. As colunas codificadas incluem "UEN", "Veiculo", "Cliente", "Origem", "Segmento", "Setor" e "Mês/ano".

In [ ]:
# Codifica a coluna 'UEN'
df['UEN'] = df['UEN'].fillna('')
df['UEN_encoded'] = le.fit_transform(df['UEN'])
df[['UEN','UEN_encoded']]

In [ ]:
#Codifica a coluna 'Veículo'
df['Veiculo'] = df['Veiculo'].fillna('')
df['Veiculo_encoded'] = le.fit_transform(df['Veiculo'])
df[['Veiculo','Veiculo_encoded']]


In [ ]:
#Codifica a coluna 'Cliente'
df['Cliente'] = df['Cliente'].fillna('')
df['Cliente_encoded'] = le.fit_transform(df['Cliente'])
df[['Cliente','Cliente_encoded']]

In [ ]:
#Codifica a coluna 'Origem'
df['Origem'] = df['Origem'].fillna('')
df['Origem_encoded'] = le.fit_transform(df['Origem'])
df[['Origem','Origem_encoded']].drop_duplicates()

In [ ]:
#Codifica a coluna 'Segmento'
df['Segmento'] = df['Segmento'].fillna('')
df['Segmento_encoded'] = le.fit_transform(df['Segmento'])
df[['Segmento','Segmento_encoded']].drop_duplicates()

In [ ]:
#Codifica a coluna 'Setor'
df['Setor'] = df['Setor'].fillna('')
df['Setor_encoded'] = le.fit_transform(df['Setor'])
df[['Setor','Setor_encoded']]

A codificação da coluna 'Mês/ano' é um pouco diferente das outras pois exige alguns passos a mais para se adequar aos parâmetros de codificação dos métodos utilizados

In [63]:
# Mapeamento manual dos meses em português, o métedo de encoding só funciona corretamente quando os meses estão em inglês
meses = {'jan.': 'Jan', 'fev.': 'Feb', 'mar.': 'Mar', 'abr.': 'Apr',
         'mai.': 'May', 'jun.': 'Jun', 'jul.': 'Jul', 'ago.': 'Aug',
         'set.': 'Sep', 'out.': 'Oct', 'nov.': 'Nov', 'dez.': 'Dec'}

# Muda os meses para inglês
df['Mês/ano'] = df['Mês/ano'].replace(meses, regex=True)

In [64]:
#Converte os meses escritos para número
df['Mês/ano'] = pd.to_datetime(df['Mês/ano'], format='%b-%y', errors='coerce')

# Formata para o formato esperado de apresentação
df['Mês/ano'] = df['Mês/ano'].dt.strftime('%Y%m')

In [65]:
# Renomeaia o nome da coluna para se adequar a um padrão de escrita;
df.rename(columns={'Mês/ano': 'Ano/mês'}, inplace=True)

Nesta parte, as colunas que passaram por codificação serão removidas do dataframe.

In [ ]:
excluir_colunas = ['UEN', 'Veiculo', 'Cliente', 'Origem', 'Segmento', 'Setor', 'CONCATENAR']
substituir_colunas = {
    'UEN_encoded': 'UEN',
    'Veiculo_encoded': 'Veiculo',
    'Cliente_encoded': 'Cliente',
    'Origem_encoded': 'Origem',
    'Segmento_encoded': 'Segmento',
    'Setor_encoded': 'Setor',
}

#Exclui as colunas que foram definidas na lista
df = df.drop(columns=excluir_colunas)
df = df.rename(columns=substituir_colunas)
df

In [ ]:
#Assim ficaram as colunas e as codificações desse 'data frame'
df.head(5)

#### Base de dados de meta/ocupação:

Esta é a página de **Meta** da base de dados, foram identificadas colunas com variáveis categóricas que necessitavam de codificação. As colunas codificadas incluem "Uen", "Mês" e "Mês/ano".

In [ ]:
#Codificando variáveis para que elas fiquem codificadas iguais a numeração real dos meses do ano 
df_meta['Mês'] = df_meta['Mês'].fillna('')

df_meta['mes_encoded'] = (df_meta['Mês'].replace('Janeiro', 1)
                                     .replace('Fevereiro', 2)
                                     .replace('Março', 3)
                                     .replace('Abril', 4)
                                     .replace('Maio', 5)
                                     .replace('Junho', 6)
                                     .replace('Julho', 7)
                                     .replace('Agosto', 8)
                                     .replace('Setembro', 9)
                                     .replace('Outubro', 10)
                                     .replace('Novembro', 11)
                                     .replace('Dezembro', 12))

df_view = df_meta['Mês'].drop_duplicates()
df_view

In [ ]:
#Substitui os valores vazios por strings vazias
df_meta['Uen'] = df_meta['Uen'].fillna('')

#Define exatamente quais variáveis serão trocadas por quais números
df_meta['Uen_encoded'] = (df_meta['Uen'].replace('Digital',0)
                                              .replace('Rádio',1)
                                              .replace('Televisão',2))

#mostra a equivalência entre a coluna normal e a codificada
df_view2 = df_meta[['Uen_encoded','Uen']].drop_duplicates()
df_view2.sort_values(by='Uen_encoded', ascending=True)

In [70]:
#Muda o nome da coluna para padronização
df_meta.rename(columns={'Mês/ano': 'Ano/mês'}, inplace=True)

#muda os nomes dos meses para inglês
df_meta['Ano/mês'] = df_meta['Ano/mês'].replace(meses, regex=True)

In [71]:
#Conversão dos meses escritos para número
df_meta['Ano/mês'] = pd.to_datetime(df_meta['Ano/mês'], format='%b-%y', errors='coerce')

# Formata para o formato de apresentação do número esperado
df_meta['Ano/mês'] = df_meta['Ano/mês'].dt.strftime('%Y%m')

Selecionando as colunas que serão excluídas da tabela.

In [ ]:
substituir_colunas = {
    'Uen_encoded': 'UEN',
    'mes_encoded': 'Mes'
}

#lista de colunas que serão excluídas
meta_excluir_colunas = ['Mês', 'Uen']

#retira as colunas não codificadas
df_meta = df_meta.drop(columns=meta_excluir_colunas)

df_meta = df_meta.rename(columns=substituir_colunas)

#Primeiras 5 linhas do resultado final:
df_meta.head()

Esta é a parte de **Ocupação** da base de dados. As colunas codificadas incluem "Uen", "Mês" e "Mês/ano".

In [ ]:
df_ocup['Veículo'].unique()

In [ ]:
#Codificando variáveis para que elas sejam atribuídas a um número inteiro para o modelo conseguir interpretar elas.
df_ocup['Veículo'] = df_ocup['Veículo'].fillna('')

df_ocup['Veículo_encoded'] = (df_ocup['Veículo'].replace('CBN VITORIA', 0)
                                     .replace('GAZETA FM LINHARES', 1)
                                     .replace('GAZETA FM VITORIA', 2)
                                     .replace('LITORAL FM', 5)
                                     .replace('LITORAL FM NOROESTE', 6)
                                     .replace('LITORAL FM NORTE', 7)
                                     .replace('LITORAL FM SUL', 8)
                                     .replace('RÁDIO MIX VITÓRIA', 11)
                                     .replace('TV GAZETA', 12)
                                     .replace('TV GAZETA NOROESTE', 13)
                                     .replace('TV GAZETA NORTE', 14)
                                     .replace('TV GAZETA SUL', 15))
df_ocup_unique = df_ocup[['Veículo', 'Veículo_encoded']].drop_duplicates(subset=['Veículo', 'Veículo_encoded'])

# Exibe o DataFrame resultante sem duplicatas
df_ocup_unique

In [ ]:
df_ocup = df_ocup.drop('Veículo', axis=1)
substituir_colunas = {'Veículo_encoded': 'Veiculo'}
df_ocup = df_ocup.rename(columns=substituir_colunas)
df_ocup

#### Base de dados econômicos

Na base de dados referentes aos Dados Econômicos, foi identificada a coluna de "Mês" que necessita de codificação. 

In [76]:
# Muda os meses para inglês
df_dados_economicos['Mês'] = df_dados_economicos['Mês'].replace(meses, regex=True)

In [77]:
#Converte os meses escritos para número
df_dados_economicos['Mês'] = pd.to_datetime(df_dados_economicos['Mês'], format='%b-%y', errors='coerce')

# Formata para o formato esperado de apresentação
df_dados_economicos['Mês'] = df_dados_economicos['Mês'].dt.strftime('%Y%m')

In [78]:
# Renomeaia o nome da coluna para se adequar a um padrão de escrita;
df_dados_economicos.rename(columns={'Mês': 'Ano/mês'}, inplace=True)

In [ ]:
#Primeiras 5 linhas do resultado final:
df_dados_economicos.head()

#### Base de dados de audiência:

Na base de dados de audiência, existem alguns processos que precisam ser realizados antes da codificação de variáveis, para que cada registro dessa tabela relacione período, veículo e audiência.

In [ ]:
# Reformula o formato da tabela, de modo que siga o mesmo formato das outras tabelas tratadas
df_audiencia = pd.melt(df_audiencia, id_vars=['PERIODO'], var_name='VEICULO', value_name='AUDIENCIA')

df_audiencia

In [ ]:
# Converte o registro dos períodos para formatação em inglês
df_audiencia['PERIODO'] = df_audiencia['PERIODO'].replace(meses, regex=True)

df_audiencia

In [ ]:
#Converte os meses escritos para número
df_audiencia['PERIODO'] = pd.to_datetime(df_audiencia['PERIODO'], format='%b-%y', errors='coerce')

df_audiencia

In [ ]:
# Formata para o formato esperado de apresentação
df_audiencia['PERIODO'] = df_audiencia['PERIODO'].dt.strftime('%Y%m')

df_audiencia

In [ ]:
# Formata o tipo de dados da variável 'PERIODO' para número
df_audiencia['PERIODO'] = df_audiencia['PERIODO'].astype(int)

df_audiencia.dtypes

In [ ]:
# Codifica os veículos da tabela
df_audiencia['VEICULO'] = df_audiencia['VEICULO'].fillna('')
df_audiencia['VEICULO_encoded'] = le.fit_transform(df_audiencia['VEICULO'])
df_audiencia[['VEICULO','VEICULO_encoded']]

In [ ]:
# Mantém apenas os valores codificados
df_audiencia['VEICULO'] = df_audiencia['VEICULO_encoded']
df_audiencia.drop('VEICULO_encoded', axis=1)

### 🧩 Concatenação de tabelas

In [87]:
# Faz com que todas as colunas de cada tabela sejam mostrados
pd.set_option('display.max_columns', None)

In [88]:
#Define duas variáveis diferentes que irão guardar apenas os dados da regiões específicas, sendo Brasil ou Espirto Santo 
df_pmc_es = df_dados_economicos[df_dados_economicos['Brasil e Unidade da Federação'] == 'Espírito Santo']
df_pmc_br = df_dados_economicos[df_dados_economicos['Brasil e Unidade da Federação'] == 'Brasil']


In [89]:
#Muda o nome das colunas para que elas se adequem a nova organização -> região Brasil
df_pmc_br = df_pmc_br.rename(columns={'pmc_indice': 'pmc_indice_Brasil', 'pmc_indice_sazonal': 'pmc_indice_sazonal_Brasil', 'pmc_variacao': 'pmc_variacao_Brasil'})

In [90]:
#Muda o nome das colunas para que elas se adequem a nova organização -> região Espirito Santo
df_pmc_es = df_pmc_es.rename(columns={'pmc_indice': 'pmc_indice_ES', 'pmc_indice_sazonal': 'pmc_indice_sazonal_ES', 'pmc_variacao': 'pmc_variacao_ES'})

In [91]:
#adiciona as colunas que contém pmc na tabela principal seguindo a distinção de Ano/mês:
df_combinedES = pd.merge(df, df_pmc_es, on='Ano/mês', how='inner', validate='many_to_one')

In [92]:
#Combina ambas as tabelas com as colunas atualizadas em uma nova tabela atualizada:
df_combined_total = pd.merge(df_combinedES , df_pmc_br, on='Ano/mês', how='inner', validate='many_to_one')

In [93]:
#Exclui as colunas duplicadas ou que não serão mais utilizadas na nova tabela
df = df_combined_total.drop(['Brasil e Unidade da Federação_y', 'Brasil e Unidade da Federação_x', 'Tipos de índice_y', 'Tipos de índice_x'], axis=1)

In [ ]:
#Mostra as um pouco de como ficou a nova tabela com suas colunas:
df.head()

## 🧪 Exploração dos dados pós realização de pré-processamento

### 📊 Estatística Descritiva 

&nbsp;&nbsp;&nbsp;&nbsp;Nessa etapa, após a realização do pré-processamento, transformação e normalização das variáveis numéricas, é possível visualizar gráficos e visualizar as estatísticas descritivas das colunas de modo mais assertivo.

In [ ]:
# Análise descritiva da base de dados principal
df.describe()

In [ ]:
# Análise descritiva da base de dados de ocupação
df_ocup.describe()

In [ ]:
# Análise descritiva da base de dados de meta
df_meta.describe()

In [ ]:
# Análise descritiva da base de dados de audiência
df_audiencia.describe()

In [ ]:
# Análise descritiva da base de dados ecomômicos
df_dados_economicos.describe()

### 📈 Gráficos para Visualização

In [ ]:
#adcionado
df_hist = df.drop(['Ano', 'Mês'], axis=1)
df_hist.hist(bins=30, figsize=(20,15))
plt.show()

In [ ]:
# Matriz de correlação
numericas_colunas = ['Ano', 'Mês', 'Ano/mês', 'Vl Liquido Final', 'Vl Bruto', 'VL Tabela', 'Desconto R$', 'Desc %', 'IPCA ES', 'IPCA BR', 'Taxa Ac. TRI % PIB', 'pmc_indice_ES', 'pmc_indice_sazonal_ES', 'pmc_variacao_ES', 'pmc_indice_Brasil','pmc_indice_sazonal_Brasil', 'pmc_variacao_Brasil']
corr_df = df[numericas_colunas]
corr_matrix = corr_df.corr()


# Ajusta o tamanho do gráfico e a formatação das anotações
plt.figure(figsize=(15, 8))  # Ajusta o tamanho da figura
sns.heatmap(corr_matrix, annot=True, fmt=".3f", cmap='coolwarm', annot_kws={"size": 10})  # fmt=".4f" para limitar a 4 casas decimais 
plt.show()

In [ ]:
df_veiculo_grafico = df.groupby('Veiculo')['Vl Liquido Final'].sum().reset_index()
df_veiculo_grafico.sort_values(by='Vl Liquido Final', ascending=False)

## 💡 Hipóteses

&nbsp;&nbsp;&nbsp;&nbsp;As hipóteses são suposições criadas sobre os dados, com base na análise exploratória realizada inicialmente. Elas representam possíveis relações, padrões ou comportamentos que esperamos encontrar nos dados após o pré-processamento. Apresenta-se abaixo as hipóteses desenvolvidas a partir das observações iniciais, seguidas dos gráficos e análises que visam comprovar sua veracidade.

- Hipótese 1 - Correlação entre Condições Econômicas e Faturamento: Variáveis macroeconômicas, como a inflação ou PIB regional, estão correlacionadas com o faturamento dos veículos de comunicação, refletindo a capacidade de investimento dos anunciantes, afetando diretamente a organização.

In [ ]:
# Matriz de correlação
numericas_colunas = ['Ano', 'Mês', 'Ano/mês', 'Vl Liquido Final', 'Vl Bruto', 'VL Tabela', 'Desconto R$', 'Desc %', 'IPCA ES', 'IPCA BR', 'Taxa Ac. TRI % PIB', 'pmc_indice_ES', 'pmc_indice_sazonal_ES', 'pmc_variacao_ES', 'pmc_indice_Brasil','pmc_indice_sazonal_Brasil', 'pmc_variacao_Brasil']
corr_df = df[numericas_colunas]
corr_matrix = corr_df.corr()


# Ajusta o tamanho do gráfico e a formatação das anotações
plt.figure(figsize=(15, 8))  # Ajusta o tamanho da figura
sns.heatmap(corr_matrix, annot=True, fmt=".3f", cmap='coolwarm', annot_kws={"size": 10})  # fmt=".4f" para limitar a 4 casas decimais 
plt.show()

- Hipótese 2 - Sazonalidade Impacta o Faturamento: O faturamento da Rede Gazeta varia significativamente conforme as estações do ano, meses ou datas comemorativas. Por exemplo, o faturamento tende a aumentar durante o período de festas de fim de ano devido ao maior investimento em publicidade.

In [ ]:
df_grouped = df[['VL Tabela', 'Vl Bruto', 'Vl Liquido Final', 'Mês']].groupby('Mês').mean().reset_index()

plt.figure(figsize=(10, 6))

sns.lineplot(x='Mês', y='VL Tabela', data=df_grouped, marker='o', label='VL Tabela')
sns.lineplot(x='Mês', y='Vl Bruto', data=df_grouped, marker='o', label='Vl Bruto')
sns.lineplot(x='Mês', y='Vl Liquido Final', data=df_grouped, marker='o', label='Vl Liquido Final')

plt.title('Sazonalidade por Mês')
plt.ylabel('Valores')
plt.xlabel('Mês')
plt.legend(title='Legenda')
plt.show()

- Hipótese 3 - Origem Geográfica da Venda e o Faturamento: A origem geográfica da venda (cidade específica do Espírito Santo ou mercado nacional) influencia significativamente o faturamento da Rede Gazeta. Vendas provenientes do mercado nacional geram maior faturamento em comparação com vendas realizadas em cidades do Espírito Santo, devido ao maior alcance e potencial de investimento de clientes de fora do estado. Dessa forma, pode-se prever a proporção entre clientes do mercado nacional e regional.

In [ ]:
barplot_columns = ['VL Tabela', 'Vl Bruto', 'Desc %', 'Vl Liquido Final']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12), sharex=True)

for i, barplot_column in enumerate(barplot_columns):
    row = i // 2
    col = i % 2  
    sns.barplot(x='Origem', y=barplot_column, data=df, estimator=sum, ax=axes[row, col])
    axes[row, col].set_title(f'Soma de {barplot_column} por Origem')
    axes[row, col].set_xlabel('Origem')
    axes[row, col].set_ylabel(f'Soma de {barplot_column}')
    axes[row, col].tick_params(axis='x', rotation=70)

plt.tight_layout()
plt.show()

## 📈 Gráficos

&nbsp;&nbsp;&nbsp;&nbsp;Nesta seção serão apresentados e documentados todos os gráficos produzidos a partir da análise exploratória dos dados, os quais são essenciais para entendimento visual acerca dos dados fornecidos, bem como para confirmação ou negação das hipóteses levantadas.

### Base de dados principal

&nbsp;&nbsp;&nbsp;&nbsp;Aqui serão disponibilizados os gráficos que se relacionam, isto é, que foram formulados a partir das análises descritivas feitas sobre a base de dados principal.

#### Diagrama de caixa

&nbsp;&nbsp;&nbsp;&nbsp;Um diagrama de caixa é uma representação gráfica que resume a distribuição de um conjunto de dados através de cinco principais estatísticas: mínimo, primeiro quartil (Q1), mediana, terceiro quartil (Q3) e máximo. Ele visualiza a dispersão e a assimetria dos dados, destacando a presença de *outliers* e a concentração dos dados em torno da mediana.

In [ ]:
# Diagrama de caixa personalizado
df_bp = df_hist.drop('Desc %', axis=1)
colunas_categoricas = ['Ano/mês', 'Veiculo', 'Cliente', 'Origem', 'Segmento', 'Setor', 'UEN']
colunas_financeiras = ['VL Tabela', 'Desconto R$', 'Vl Bruto', 'Vl Liquido Final']
colunas_indicadores_econ = ['IPCA ES', 'IPCA BR', 'Taxa Ac. TRI % PIB', 'pmc_indice_ES', 'pmc_indice_sazonal_ES', 'pmc_variacao_ES', 'pmc_indice_Brasil', 'pmc_indice_sazonal_Brasil', 'pmc_variacao_Brasil']
df_bp

In [ ]:
plt.figure(figsize=(16, 10))
sns.boxplot(data=df_bp[colunas_indicadores_econ])
plt.title('Distribuição dos Indicadores Econômicos')
plt.ylabel('Valores dos Indicadores (%)')
plt.xlabel('Indicadores Econômicos')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
sns.boxplot(data=df_bp[colunas_financeiras])
plt.title('Distribuição das Colunas Financeiras')
plt.ylabel('Valores das Colunas Financeiras (em base logariítimica)')
plt.xlabel('Colunas Financeiras')
plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
sns.boxplot(data=df_bp['Vl Liquido Final s/ normalização'])
plt.title('Distribuição dos Valores Líquidos Finais de cada contrato')
plt.ylabel('Valores Líquidos (em milhões)')
plt.xlabel('Valor Líquido Final sem normalização')
plt.show()

#### Histogramas

&nbsp;&nbsp;&nbsp;&nbsp;Um histograma é uma representação gráfica que mostra a distribuição de um conjunto de dados numéricos ao dividir os dados em intervalos e contar a frequência de valores dentro de cada intervalo. No gráfico, os intervalos são representados por barras, cuja altura indica o número de ocorrências em cada intervalo.

In [ ]:
# Histogramas
df_hist.hist(bins=30, figsize=(20,15))
plt.show()

In [ ]:
# Gráfico de histograma
columns_sct = ['VL Tabela', 'Vl Bruto', 'Vl Liquido Final']

n = len(columns_sct)
fig, axes = plt.subplots(nrows=1, ncols=n, figsize=(5 * n, 6), sharex=True, sharey=True)

if n == 1:
    axes = [axes]

for i, column_sct in enumerate(columns_sct):
    sns.histplot(df_hist[column_sct], kde=True, stat="density", ax=axes[i])
    axes[i].set_title(f'Histograma de {column_sct}')
    axes[i].set_xlabel(f'{column_sct}')
    axes[i].set_ylabel('Densidade')

plt.tight_layout()
plt.show()

#### Gráfico de linhas

&nbsp;&nbsp;&nbsp;&nbsp;Um gráfico de linhas é uma representação visual que conecta pontos de dados individuais ao longo de uma linha contínua, geralmente usada para mostrar a evolução de uma variável ao longo do tempo. No eixo horizontal (x), são plotados os valores de uma variável independente, enquanto no eixo vertical (y), são exibidos os valores da variável dependente.

In [ ]:
# Line plot
df_grouped = df[['VL Tabela', 'Vl Bruto', 'Vl Liquido Final', 'Mês']].groupby('Mês').mean().reset_index()

plt.figure(figsize=(10, 6))

sns.lineplot(x='Mês', y='VL Tabela', data=df_grouped, marker='o', label='VL Tabela')
sns.lineplot(x='Mês', y='Vl Bruto', data=df_grouped, marker='o', label='Vl Bruto')
sns.lineplot(x='Mês', y='Vl Liquido Final', data=df_grouped, marker='o', label='Vl Liquido Final')

plt.title('Sazonalidade por Mês')
plt.ylabel('Valores')
plt.xlabel('Mês')
plt.legend(title='Legenda')
plt.show()


#### Gráficos de barras

&nbsp;&nbsp;&nbsp;&nbsp;Um gráfico de barras é uma representação visual que utiliza barras retangulares para comparar valores entre diferentes categorias ou grupos. No gráfico, o comprimento de cada barra é proporcional ao valor que ela representa. O eixo horizontal exibe as categorias, enquanto o outro eixo mostra as frequências ou valores associados.

In [ ]:
# Gráficos de barras
barplot_columns = ['VL Tabela', 'Vl Bruto', 'Desc %', 'Vl Liquido Final']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12), sharex=True)

for i, barplot_column in enumerate(barplot_columns):
    row = i // 2
    col = i % 2  
    sns.barplot(x='Origem', y=barplot_column, data=df, estimator=sum, ax=axes[row, col])
    axes[row, col].set_title(f'Soma de {barplot_column} por Origem')
    axes[row, col].set_xlabel('Origem')
    axes[row, col].set_ylabel(f'Soma de {barplot_column}')
    axes[row, col].tick_params(axis='x', rotation=70)

plt.tight_layout()
plt.show()

#### Matriz de correlação

&nbsp;&nbsp;&nbsp;&nbsp;Uma matriz de correlação é uma tabela que mostra os coeficientes de correlação entre todas as possíveis combinações de variáveis em um conjunto de dados. Cada célula na matriz exibe um valor que indica a força e a direção da relação linear entre duas variáveis, variando de -1 (correlação negativa perfeita) a 1 (correlação positiva perfeita), com 0 indicando ausência de correlação.

In [ ]:
# Matriz de correlação
numericas_colunas = ['Ano', 'Mês', 'Ano/mês', 'Vl Liquido Final', 'Vl Bruto', 'VL Tabela', 'Desconto R$', 'Desc %', 'IPCA ES', 'IPCA BR', 'Taxa Ac. TRI % PIB', 'pmc_indice_ES', 'pmc_indice_sazonal_ES', 'pmc_variacao_ES', 'pmc_indice_Brasil','pmc_indice_sazonal_Brasil', 'pmc_variacao_Brasil']
corr_df = df[numericas_colunas]
corr_matrix = corr_df.corr()


# Ajusta o tamanho do gráfico e a formatação das anotações
plt.figure(figsize=(15, 8))  # Ajusta o tamanho da figura
sns.heatmap(corr_matrix, annot=True, fmt=".4f", cmap='coolwarm', annot_kws={"size": 10})  # fmt=".4f" para limitar a 4 casas decimais 
plt.show()

## 🧠 Modelagem

### ⚙️ Modelos Avaliados

#### Árvore de Decisão

O Random Forest é uma modelo supervisionado baseada em árvores de decisão que combina diversas árvores para melhorar a precisão das previsões e reduzir o overfitting. Ao longo desse bloco de código, foi feita a escolha de features, separação do dataframe em conjuntos de treino, teste e validação e, por fim, é exibido as métricas.

In [113]:
x = df[['Mês', 'Veiculo', 'Origem', 'Segmento', 'Setor']] #df apenas com as features desejadas
y = df['Vl Liquido Final'] #Coluna desejada para a previsão

# Separando os dados em treino (60%), validação (20%) e teste (20%)
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=42)

# Dividindo o conjunto temporário (X_temp) em validação e teste
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Inicializando o modelo Random Forest com hiperparâmetros padrões
rf_model = RandomForestRegressor(n_estimators=200, random_state=42, max_depth=30, min_samples_leaf=2, min_samples_split=10, max_features='sqrt')

# Treinando o modelo com os dados de treino (somente colunas categóricas)
rf_model.fit(X_train, y_train)

# Fazendo previsões no conjunto de validação
y_val_pred = rf_model.predict(X_val)

# Avaliando o desempenho no conjunto de validação
val_r2 = r2_score(y_val, y_val_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
val_mae = mean_absolute_error(y_val, y_val_pred)
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)*100

# Exibindo os resultados do conjunto de validação
print(f"Validação - R²: {val_r2:.4f}")
print(f"Validação - RMSE: {val_rmse:.2f}")
print(f"Validação - MAE: {val_mae:.2f}")
print(f"Validação - MAPE: {val_mape:.2f}%")

In [ ]:
y_test_pred = rf_model.predict(X_test)
# Avaliando o modelo no conjunto de teste
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

# Exibindo os resultados do conjunto de teste
print(f"Teste - R²: {test_r2:.4f}")
print(f"Teste - RMSE: {test_rmse:.2f}")
print(f"Teste - MAE: {test_mae:.2f}")

#### Regressão Linear Múltipla

No modelo de regressão linear múltipla, foi necessário criar um novo dataframe, agrupando as colunas com base no veículo, ano e mês. Após isso, foi aplicado a normalização em uma coluna, selecionado as features e feita a separação do dataframe em conjuntos de treino, teste e validação. Após o treinamento do modelo, foram calculadas e exibidas as métricas. Por fim, há a presença de gráficos para facilitar a visualização dos resultados do modelo.

In [ ]:
#Agrupa dados na tentativa de gerar um modelo mais preciso na predição
df_veiculoR = df.groupby(['Veiculo', 'Ano', 'Mês']).agg({
    'Vl Liquido Final': 'sum', 
    'IPCA ES': 'mean',                  
    'IPCA BR': 'mean',                 
    'Taxa Ac. TRI % PIB': 'mean',    
    'Vl Liquido Final s/ norm': 'sum', 
    'UEN': 'first',                    
    'pmc_indice_ES': 'mean',            
    'pmc_indice_sazonal_ES': 'mean',    
    'pmc_variacao_ES': 'mean',         
    'pmc_indice_Brasil': 'mean',       
    'pmc_indice_sazonal_Brasil': 'mean',
    'pmc_variacao_Brasil': 'mean'       
}).reset_index()

# Exibir o DataFrame resultante
df_veiculoR

In [117]:
df_veiculoR['Vl Liquido Final normalizado'] = np.log1p(df_veiculoR['Vl Liquido Final s/ norm'])

In [118]:
xR = df_veiculoR[['Ano', 'Mês', 'Veiculo', 'Taxa Ac. TRI % PIB', 'pmc_indice_sazonal_ES', 'UEN']] #df apenas com as features desejadas
yR = df_veiculoR['Vl Liquido Final normalizado'] #Coluna desejada para a previsão

# Separando os dados em treino (60%), validação (20%) e teste (20%)
X_trainR, X_tempR, y_trainR, y_tempR = train_test_split(xR, yR, test_size=0.4, random_state=42)

# Dividindo o conjunto temporário (X_temp) em validação e teste
X_valR, X_testR, y_valR, y_testR = train_test_split(X_tempR, y_tempR, test_size=0.5, random_state=42)

In [ ]:
# Relaciona a função da  regressão linear Múltipla com uma variável que será aplicada ao projeto
modelR = LinearRegression()

#Treina o modelo com os dados de treino
modelR.fit(X_trainR, y_trainR)

In [120]:
# Fazer previsões com o conjunto de teste, ja utilizando o algoritmo que acabou de ser treinado
y_predR = modelR.predict(X_testR)

In [ ]:
#Avalia a performance do modelo

# Calcula o MSE mede a média das diferenças absolutas entre os valores reais e previstos. 
# Ele indica, em média, o quanto os valores previstos se desviam dos valores reais.-> quanto menor o valor presvisto, melhor é a qualidade da previsão 
mseR = mean_squared_error(y_testR, y_predR)
print(f'MSE: {mseR}')

# Calcula o R^2 é uma métrica que indica o quão bem o modelo explica a variabilidade dos dados. Ele varia de 0 a 1 -> quanto mais próximo de 1 o resultado 
# estiver, melhor é o modelo para os dados de treinamento
r2R = r2_score(y_testR, y_predR)
print(f'R^2: {r2R}')

#Calcula o MAPE, expressa o erro médio absoluto como uma porcentagem dos valores reais, 
# o que facilita a interpretação, pois mostra o erro em termos percentuais. -> quanto menor a porcentagem, melhor
mapeR = np.mean(np.abs((y_testR - y_predR) / y_testR)) * 100
print(f"Mean Absolute Percentage Error (MAPE): {mapeR}")

In [ ]:
# Mostra os coeficientes das variáveis independentes
print(f'Coeficientes: {modelR.coef_}')

# Mosta a Interceptação
print(f'Interceptação: {modelR.intercept_}')

In [ ]:
# Mostra um gráfico do valor real contra o ideal do modelo
plt.scatter(y_testR, y_predR, color='blue', label='Valores Previstos')
plt.plot([y_testR.min(), y_testR.max()], [y_testR.min(), y_testR.max()], color='red', lw=1, label='Linha de Referência/Modelo Ideal', linestyle='dashed')
plt.xlabel('Valores Reais (log(1+x))')
plt.ylabel('Previsões (log(1+x))')
plt.title('Valores Reais vs Previsões Estatísticas')
plt.legend()  # Mostra a legenda
plt.grid(linewidth = 0.2)
plt.show()

In [ ]:
# Exibi os coeficietes -> esses coeficientes mostram o quão diretamente relacionados cada tópico está para o resultado final da predição
coef_dfR = pd.DataFrame(modelR.coef_, xR.columns, columns=['Coeficientes'])

print(coef_dfR)

#### SARIMA

In [ ]:
coluna = 'Veiculo'
indice = 0

veiculos_nomes = {
    0: 'CBN VITORIA',
    1: 'GAZETA FM LINHARES',
    2: 'GAZETA FM VITORIA',
    3: 'GAZETA PRODUCOES',
    4: 'INTERNET',
    5: 'LITORAL FM',
    6: 'LITORAL FM NOROESTE',
    7: 'LITORAL FM NORTE',
    8: 'LITORAL FM SUL',
    9: 'PORTAL G1/GE/HOME',
    10: 'PRODUCAO',
    11: 'RADIO MIX VITORIA',
    12: 'TV GAZETA',
    13: 'TV GAZETA NOROESTE',
    14: 'TV GAZETA N0RTE',
    15: 'TV GAZETA SUL',
    16: 'TV NOROESTE PRODUCOES',
    17: 'TV NORTE PRODUCOES',
    18: 'TV SUL PRODUCOES'
}

origens_nomes = {
    0: 'CH - CONTATO - CACHOEIRO',
    1: 'CO - CONTATO - COLATINA',
    2: 'LI - CONTATO - LINHARES',
    3: 'MP - MÍDIA PROGRAMÁTICA',
    4: 'RN - MERCADO NACIONAL',
    5: 'VT - CONTATO - VITÓRIA'
}

segmentos_nomes = {
    0: 'AGROPECUARIA',
    1: 'ALIMENTOS',
    2: 'BEBIDAS',
    3: 'BENS INDUSTRIAIS MAT. PRIMA',
    4: 'BRINQUEDOS E DIVERSOES',
    5: 'COMERCIO',
    6: 'CONDOMINIOS',
    7: 'EDUCACAO/MEIOS DE COMUNICACAO',
    8: 'EQUIP/MAT/ESCRIT/LOJA/ESCOLA',
    9: 'FOTO / OTICA / CINE / SOM',
    10: 'INDUSTRIA DA CONSTRUCAO',
    11: 'INTERNET',
    12: 'LIMPEZA / HIGIENE DOMESTICA',
    13: 'MEIOS DE TRANSPORTES E AFINS',
    14: 'MERCADO FINANCEIRO',
    15: 'MOVEIS E DECORACOES',
    16: 'OUTRAS RECEITAS',
    17: 'OUTROS',
    18: 'PERFUMARIA / FARMACIA',
    19: 'SERVICOS',
    20: 'SERVICOS DE TELECOMUNICACOES',
    21: 'TEXTIL / VESTUARIO',
    22: 'UTILIDADES DOMESTICAS'
}

dropdown1 = widgets.Dropdown(
    options=['Veiculo', 'Origem', 'Segmento'],
    value=None,
    description='Prever por:',
    disabled=False,
)

dropdown2 = widgets.Dropdown(
    options=[],
    value = None,
    description='Índice:',
    disabled=False,
)

def atualizar_dropdown2(opcao):
    global coluna
    coluna = opcao['new']
    
    if coluna == 'Veiculo':
        dropdown2.options = [(v, k) for k, v in veiculos_nomes.items()]
    elif coluna == 'Origem':
        dropdown2.options = [(v, k) for k, v in origens_nomes.items()]
    elif coluna == 'Segmento':
        dropdown2.options = [(v, k) for k, v in segmentos_nomes.items()]
    
    dropdown2.value = None
    
def on_value_change_dropdown2(opcao):
    global indice
    indice = opcao['new']

dropdown1.observe(atualizar_dropdown2, names='value')
dropdown2.observe(on_value_change_dropdown2, names='value')

display(dropdown1)
display(dropdown2)


In [ ]:
print(f'Selecionado: {coluna} ({indice})\nPor favor, execute as células abaixos novamentes para o resultado atualizado')

In [127]:
if coluna == 'Veiculo' or coluna == 'Origem' or coluna == 'Segmento':
    df_temp = df.sort_values(by=coluna)
    df_coluna = df_temp.groupby(coluna)

dfs_novos = []
for item_unico in df_temp[coluna].unique():
    dfs_novos.append(df_coluna.get_group(item_unico))

In [128]:
if coluna == 'Veiculo':
    indice_max = df['Veiculo'].nunique()-1
elif coluna == 'Origem':
    indice_max = df['Origem'].nunique()-1
elif coluna == 'Segmento':
    indice_max = df['Segmento'].nunique()-1

def escolher_df (df_escolhido):
    if df_escolhido >= 0 or df_escolhido <= indice_max:
        return dfs_novos[df_escolhido]

In [ ]:
print(f'Índice máximo veículo:', df['Veiculo'].nunique()-1)
print(f'Índice máximo origem:', df['Origem'].nunique()-1)
print(f'Índice máximo segmento:', df['Segmento'].nunique()-1)

In [ ]:
df_sarima = escolher_df(indice)
df_sarima = df_sarima[['Ano/mês', 'Vl Liquido Final']]
df_sarima

In [ ]:
df_sarima['Vl Liquido Final'] = df_sem_escala['Vl Liquido Final']
df_sarima

In [ ]:
df_sarima['Ano/mês'] = pd.to_datetime(df_sarima['Ano/mês'], format='%Y%m')
df_sarima.set_index('Ano/mês', inplace=True)
df_sarima = df_sarima.sort_index()

df_sarima

In [ ]:
df_sarima = df_sarima.groupby(df_sarima.index).sum()
df_sarima = np.log1p(df_sarima)
df_sarima

In [ ]:
df_sarima = df_sarima.asfreq('MS')
df_sarima

In [ ]:
df_sarima = df_sarima.fillna(method='ffill')
df_sarima

In [ ]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

seasonal_pdq = [(x[0], x[1], x[2], 6) for x in pdq]

results = []

y = df_sarima

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            model = SARIMAX(y, order=param, seasonal_order=param_seasonal, enforce_stationarity=False, enforce_invertibility=False)
            result = model.fit()
            results.append((param, param_seasonal, result.aic))
        except Exception as e:
            print(f'Erro ao ajustar ARIMA{param}x{param_seasonal}12: {e}')
            continue

if results:
    best_params = sorted(results, key=lambda x: x[2])[0]
    print(f'Melhor combinação de parâmetros: {best_params[0]}x{best_params[1]}12 - AIC: {best_params[2]}')
else:
    print("Nenhuma combinação de parâmetros conseguiu ajustar o modelo.")

In [139]:
train_size = int(len(df_sarima) * 0.8)
train, test = df_sarima['Vl Liquido Final'][:train_size], df_sarima['Vl Liquido Final'][train_size:]

In [140]:
model = SARIMAX(train,
                order=best_params[0],
                seasonal_order=best_params[1],
                enforce_stationarity=False,
                enforce_invertibility=False)

In [ ]:
sarima_model = model.fit(disp=False)

sarima_model.summary()

In [ ]:
predictions = sarima_model.get_forecast(steps=len(test))
forecasted_values = predictions.predicted_mean
pred_ci = predictions.conf_int()

ax = test.plot(label='Real', figsize=(10, 6))
predictions.predicted_mean.plot(ax=ax, label='Previsões', alpha=0.7)
ax.fill_between(pred_ci.index, pred_ci.iloc[:, 0], pred_ci.iloc[:, 1], color='k', alpha=0.2)
plt.legend()
plt.show()

In [ ]:
r2 = r2_score(test, forecasted_values)*100
mae = mean_absolute_error(test, forecasted_values)
mae_real = np.expm1(mae)
mape = mean_absolute_percentage_error(test, forecasted_values)*100

print(f'R² do modelo: {r2:.2f}%')
print(f"MAE do modelo: {mae}")
print(f'MAE real: {mae_real}')
print(f"MAPE do modelo: {mape:.2f}%")

### ✅ Modelo Final - LightGBM

Por conta do excelente desempenho na capacidade de previsão de receita e por ser capaz de criar previsões de diferentes combinações, como por veículo e segmento e por veículo e origem, optamos por utilizar o modelo LightGBM como nosso modelo final. 

#### Modelo 1 - Previsão por Veículo

In [ ]:
df_veiculo = df.groupby(['Veiculo', 'Ano', 'Mês']).agg({
    'IPCA ES': 'mean',                  
    'IPCA BR': 'mean',                 
    'Taxa Ac. TRI % PIB': 'mean',    
    'Vl Liquido Final s/ normalização': 'sum', 
    'UEN': 'first',                    
    'pmc_indice_ES': 'mean',            
    'pmc_indice_sazonal_ES': 'mean',    
    'pmc_variacao_ES': 'mean',         
    'pmc_indice_Brasil': 'mean',       
    'pmc_indice_sazonal_Brasil': 'mean',
    'pmc_variacao_Brasil': 'mean'       
}).reset_index()

# Exibir o DataFrame resultante
df_veiculo

In [138]:
# Normalização da Coluna Vl Liquido após somar os valores
df_veiculo['Vl Liquido Final normalizado'] = np.log1p(df_veiculo['Vl Liquido Final s/ normalização'])

In [139]:
x_veiculo = df_veiculo[['Veiculo','Ano','Mês', 'Taxa Ac. TRI % PIB']] #df apenas com as features desejadas
y_veiculo = df_veiculo['Vl Liquido Final normalizado'] #Coluna alvo

# Separando os dados em treino (60%), validação (20%) e teste (20%)
X_train_veiculo, X_temp_veiculo, y_train_veiculo, y_temp_veiculo = train_test_split(x_veiculo, y_veiculo, test_size=0.4, random_state=42)

# Dividindo o conjunto temporário (X_temp) em validação e teste
X_val_veiculo, X_test_veiculo, y_val_veiculo, y_test_veiculo = train_test_split(X_temp_veiculo, y_temp_veiculo, test_size=0.5, random_state=42)

In [ ]:
train_data_veiculo = lgb.Dataset(X_train_veiculo, label=y_train_veiculo)
val_data_veiculo = lgb.Dataset(X_val_veiculo, label=y_val_veiculo)

params_veiculo = {'learning_rate': 0.06, 
          'num_leaves': 126, 
          'feature_fraction': 0.7, 
          'bagging_fraction': 0.9, 
          'bagging_freq': 10, 
          'n_estimators': 790,
          'seed': 42}

callbacks = [lgb.early_stopping(stopping_rounds=100)]
model_veiculo = lgb.train(params_veiculo, 
                  train_data_veiculo, 
                  num_boost_round=1000, 
                  valid_sets=[val_data_veiculo], 
                  callbacks=callbacks)


y_pred_veiculo = model_veiculo.predict(X_test_veiculo, num_iteration=model_veiculo.best_iteration)

In [ ]:
# Cálculo das métricas para o modelo
rmse_veiculo = mean_squared_error(y_test_veiculo, y_pred_veiculo, squared=False)
r2_veiculo = r2_score(y_test_veiculo, y_pred_veiculo)
mae_veiculo = mean_absolute_error(y_test_veiculo, y_pred_veiculo)
mape_veiculo = mean_absolute_percentage_error(y_test_veiculo, y_pred_veiculo)

# Exibição das métricas para o modelo
print(f'Teste - RMSE: {rmse_veiculo:.4f}')
print(f'Teste - R²: {r2_veiculo:.4f}')
print(f'Teste - MAE: {mae_veiculo:.4f}')
print(f'Teste - MAPE: {mape_veiculo:.4f}')

##### Modelo 1 com otimização dos hiperparâmetros

In [ ]:
# Definir o nível de log do Optuna como WARNING para evitar impressões detalhadas
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Ignorar avisos
warnings.filterwarnings("ignore")

# Definir a função objetivo para a otimização
def objective_veiculo_opt(trial):
    params_veiculo_opt = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.07),
        'num_leaves': trial.suggest_int('num_leaves', 31, 130),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.8),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        'verbose': -1,  # Desativar logs desnecessários
        'seed': 42  
    }
    
    train_data_veiculo_opt = lgb.Dataset(X_train_veiculo, label=y_train_veiculo)
    val_data_veiculo_opt = lgb.Dataset(X_val_veiculo, label=y_val_veiculo)
    
    model_veiculo_opt = lgb.train(params_veiculo_opt, 
                                  train_data_veiculo_opt, 
                                  num_boost_round=1000, 
                                  valid_sets=[val_data_veiculo_opt], 
                                  callbacks=[lgb.early_stopping(stopping_rounds=100)])
    
    y_pred_veiculo_opt = model_veiculo_opt.predict(X_val_veiculo, num_iteration=model_veiculo_opt.best_iteration)
    
    mape_veiculo_opt = mean_absolute_percentage_error(y_val_veiculo, y_pred_veiculo_opt) * 100
    return mape_veiculo_opt

# Otimização
study_veiculo_opt = optuna.create_study(direction='minimize')  
study_veiculo_opt.optimize(objective_veiculo_opt, n_trials=50)

# Mostrar melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados para Veículo (Otimizado):", study_veiculo_opt.best_params)

# Treinar o modelo com os melhores hiperparâmetros
best_params_veiculo_opt = study_veiculo_opt.best_params
best_params_veiculo_opt['verbose'] = -1  # Desativar logs desnecessários

train_data_veiculo_opt = lgb.Dataset(X_train_veiculo, label=y_train_veiculo)
val_data_veiculo_opt = lgb.Dataset(X_val_veiculo, label=y_val_veiculo)

best_model_veiculo_opt = lgb.train(best_params_veiculo_opt, 
                                   train_data_veiculo_opt, 
                                   num_boost_round=1000, 
                                   valid_sets=[val_data_veiculo_opt], 
                                   callbacks=[lgb.early_stopping(stopping_rounds=100)])

# Avaliação final no conjunto de teste
y_pred_test_veiculo_opt = best_model_veiculo_opt.predict(X_test_veiculo, num_iteration=best_model_veiculo_opt.best_iteration)


In [ ]:
# Cálculo das métricas para o modelo otimizado
rmse_veiculo_opt = mean_squared_error(y_test_veiculo, y_pred_test_veiculo_opt, squared=False)
r2_veiculo_opt = r2_score(y_test_veiculo, y_pred_test_veiculo_opt)
mae_veiculo_opt = mean_absolute_error(y_test_veiculo, y_pred_test_veiculo_opt)
mape_veiculo_opt = mean_absolute_percentage_error(y_test_veiculo, y_pred_test_veiculo_opt)

# Exibição das métricas para o modelo otimizado
print('Métricas com a Otimização dos Hiperparâmetros')
print(f'Teste - RMSE: {rmse_veiculo_opt:.4f}')
print(f'Teste - R²: {r2_veiculo_opt:.4f}')
print(f'Teste - MAE: {mae_veiculo_opt:.4f}')
print(f'Teste - MAPE: {mape_veiculo_opt:.4f}')

In [ ]:
# Plotar valores reais vs previstos para o modelo otimizado
plt.figure(figsize=(12, 6))

# Garantir que os dados de teste estejam alinhados e sem reset de índices desnecessários
plt.plot(y_test_veiculo.reset_index(drop=True)[:100], label='Valores Reais', color='#6baed6', linestyle='-')
plt.plot(y_pred_test_veiculo_opt[:100], label='Valores Previstos', color='red', linestyle='--')

# Configurações do gráfico
plt.xlabel('Índice')
plt.ylabel('Valor ( ln(1+x) )')
plt.title('Análise das Predições do Modelo 1: Valores Reais vs. Valores Previstos (Primeiros 100 Dados)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Tamanho do gráfico
plt.figure(figsize=(10, 6))

# Gráfico de dispersão dos valores reais vs previstos
plt.scatter(y_test_veiculo, y_pred_test_veiculo_opt, alpha=0.6, edgecolors='k', color='blue', label='Valores reais em comparação com os preditos')

# Adicionar uma linha de referência (linha ideal onde os valores reais são iguais aos previstos)
plt.plot([y_test_veiculo.min(), y_test_veiculo.max()], [y_test_veiculo.min(), y_test_veiculo.max()], 'r--', lw=2, label='Linha de Posição Ideal')

# Adicionando a legenda
plt.legend(title="Legenda", loc="lower right", fontsize = 10)  # Adiciona a legenda no canto inferior direito

# Rótulos e título
plt.xlabel('Valores Reais ( ln(1+x) )')
plt.ylabel('Valores Preditos ( ln(1+x) )')
plt.title('Diagrama de Dispersão das Predições do Modelo 1 (Veículo)')

# Adicionar grid
plt.grid(True)

# Exibir gráfico
plt.show()

#### Modelo 2 - Previsão por Veículo e Segmento

In [ ]:
df_segmento = df.groupby(['Veiculo', 'Ano', 'Mês', 'Segmento']).agg({
    'IPCA ES': 'mean',                  
    'IPCA BR': 'mean',                 
    'Taxa Ac. TRI % PIB': 'mean',    
    'Vl Liquido Final s/ normalização': 'sum', 
    'UEN': 'first',                    
    'pmc_indice_ES': 'mean',            
    'pmc_indice_sazonal_ES': 'mean',    
    'pmc_variacao_ES': 'mean',         
    'pmc_indice_Brasil': 'mean',       
    'pmc_indice_sazonal_Brasil': 'mean',
    'pmc_variacao_Brasil': 'mean'       
}).reset_index()

# Exibir o DataFrame resultante
df_segmento

In [147]:
df_segmento['Vl Liquido Final normalizado'] = np.log1p(df_segmento['Vl Liquido Final s/ normalização'])

In [148]:
x_segmento = df_segmento[['Veiculo','Ano','Mês', 'Taxa Ac. TRI % PIB', 'Segmento']] #df apenas com as features desejadas
y_segmento = df_segmento['Vl Liquido Final normalizado'] #Coluna desejada para a previsão

# Separando os dados em treino (60%), validação (20%) e teste (20%)
X_train_segmento, X_temp_segmento, y_train_segmento, y_temp_segmento = train_test_split(x_segmento, y_segmento, test_size=0.4, random_state=42)

# Dividindo o conjunto temporário (X_temp) em validação e teste
X_val_segmento, X_test_segmento, y_val_segmento, y_test_segmento = train_test_split(X_temp_segmento, y_temp_segmento, test_size=0.5, random_state=42)

In [ ]:
train_data_segmento = lgb.Dataset(X_train_segmento, label=y_train_segmento)
val_data_segmento = lgb.Dataset(X_val_segmento, label=y_val_segmento)

params_segmento = {'learning_rate': 0.06, 
          'num_leaves': 126, 
          'feature_fraction': 0.7, 
          'bagging_fraction': 0.9, 
          'bagging_freq': 10, 
          'n_estimators': 790,
          'seed': 42}

callbacks = [lgb.early_stopping(stopping_rounds=100)]
model_segmento = lgb.train(params_segmento, 
                  train_data_segmento, 
                  num_boost_round=1000, 
                  valid_sets=[val_data_segmento], 
                  callbacks=callbacks)


y_pred_segmento = model_segmento.predict(X_test_segmento, num_iteration=model_segmento.best_iteration)

In [ ]:
# Cálculo das métricas para o modelo
rmse_segmento = mean_squared_error(y_test_segmento, y_pred_segmento, squared=False)
r2_segmento = r2_score(y_test_segmento, y_pred_segmento)
mae_segmento = mean_absolute_error(y_test_segmento, y_pred_segmento)
mape_segmento = mean_absolute_percentage_error(y_test_segmento, y_pred_segmento)

# Exibição das métricas para o modelo
print(f'Teste - RMSE: {rmse_segmento:.4f}')
print(f'Teste - R²: {r2_segmento:.4f}')
print(f'Teste - MAE: {mae_segmento:.4f}')
print(f'Teste - MAPE: {mape_segmento:.4f}')

##### Modelo 2 com otimização dos hiperparâmetros

In [ ]:
# Ignorar avisos
warnings.filterwarnings("ignore")

# definir a função objetivo para a otimização
def objective_segmento_opt(trial):
    params_segmento_opt = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.07),
        'num_leaves': trial.suggest_int('num_leaves', 31, 130),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.8),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        'verbose': -1,  # Desativar logs desnecessários
        'seed': 42  
    }
    
    train_data_segmento_opt = lgb.Dataset(X_train_segmento, label=y_train_segmento)
    val_data_segmento_opt = lgb.Dataset(X_val_segmento, label=y_val_segmento)
    
    model_segmento_opt = lgb.train(params_segmento_opt, 
                                   train_data_segmento_opt, 
                                   num_boost_round=1000, 
                                   valid_sets=[val_data_segmento_opt], 
                                   callbacks=[lgb.early_stopping(stopping_rounds=100)])
    
    y_pred_segmento_opt = model_segmento_opt.predict(X_val_segmento, num_iteration=model_segmento_opt.best_iteration)
    
    mape_segmento_opt = mean_absolute_percentage_error(y_val_segmento, y_pred_segmento_opt) * 100
    return mape_segmento_opt

# Otimização
study_segmento_opt = optuna.create_study(direction='minimize')  
study_segmento_opt.optimize(objective_segmento_opt, n_trials=50)

# Mostrar melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados para Segmento (Otimizado):", study_segmento_opt.best_params)

# Treinar o modelo com os melhores hiperparâmetros
best_params_segmento_opt = study_segmento_opt.best_params
best_params_segmento_opt['verbose'] = -1  # Desativar logs desnecessários

train_data_segmento_opt = lgb.Dataset(X_train_segmento, label=y_train_segmento)
val_data_segmento_opt = lgb.Dataset(X_val_segmento, label=y_val_segmento)

best_model_segmento_opt = lgb.train(best_params_segmento_opt, 
                                    train_data_segmento_opt, 
                                    num_boost_round=1000, 
                                    valid_sets=[val_data_segmento_opt], 
                                    callbacks=[lgb.early_stopping(stopping_rounds=100)])

# Avaliação final no conjunto de teste
y_pred_test_segmento_opt = best_model_segmento_opt.predict(X_test_segmento, num_iteration=best_model_segmento_opt.best_iteration)

In [ ]:
# Cálculo das métricas para o modelo otimizado
rmse_segmento_opt = mean_squared_error(y_test_segmento, y_pred_test_segmento_opt, squared=False)
r2_segmento_opt = r2_score(y_test_segmento, y_pred_test_segmento_opt)
mae_segmento_opt = mean_absolute_error(y_test_segmento, y_pred_test_segmento_opt)
mape_segmento_opt = mean_absolute_percentage_error(y_test_segmento, y_pred_test_segmento_opt)

# Exibição das métricas para o modelo otimizado
print('Métricas com a Otimização dos Hiperparâmetros')
print(f'Teste - RMSE: {rmse_segmento_opt:.4f}')
print(f'Teste - R²: {r2_segmento_opt:.4f}')
print(f'Teste - MAE: {mae_segmento_opt:.4f}')
print(f'Teste - MAPE: {mape_segmento_opt:.4f}')

In [ ]:
# Plotar valores reais vs previstos para o modelo otimizado
plt.figure(figsize=(12, 6))

# Garantir que os dados de teste estejam alinhados e sem reset de índices desnecessários
plt.plot(y_test_segmento.reset_index(drop=True)[:100], label='Valores Reais', color='#6baed6', linestyle='-')
plt.plot(y_pred_test_segmento_opt[:100], label='Valores Previstos', color='red', linestyle='--')

# Configurações do gráfico
plt.xlabel('Índice')
plt.ylabel('Valor ( ln(1+x) )')
plt.title('Análise das Predições do Modelo 2: Valores Reais vs. Valores Previstos (Primeiros 100 Dados)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Tamanho do gráfico
plt.figure(figsize=(10, 6))

# Gráfico de dispersão dos valores reais vs previstos
plt.scatter(y_test_segmento, y_pred_test_segmento_opt, alpha=0.6, edgecolors='k', color='blue', label='Valores reais em comparação com os preditos')

# Adicionar uma linha de referência (linha ideal onde os valores reais são iguais aos previstos)
plt.plot([y_test_segmento.min(), y_test_segmento.max()], [y_test_segmento.min(), y_test_segmento.max()], 'r--', lw=2, label='Linha de Posição Ideal')

# Adicionando a legenda
plt.legend(title="Legenda", loc="lower right", fontsize = 10)  # Adiciona a legenda no canto inferior direito

# Rótulos e título
plt.xlabel('Valores Reais ( ln(1+x) )')
plt.ylabel('Valores Preditos ( ln(1+x) )')
plt.title('Diagrama de Dispersão das Predições do Modelo 2 (Veículo e Segmento)')

# Adicionar grid
plt.grid(True)

# Exibir gráfico
plt.show()

#### Modelo 3 - Previsão por Veículo e Origem

In [ ]:
df_origem = df.groupby(['Veiculo', 'Ano', 'Mês', 'Origem']).agg({
    'IPCA ES': 'mean',                  
    'IPCA BR': 'mean',                 
    'Taxa Ac. TRI % PIB': 'mean',    
    'Vl Liquido Final s/ normalização': 'sum', 
    'UEN': 'first',                    
    'pmc_indice_ES': 'mean',            
    'pmc_indice_sazonal_ES': 'mean',    
    'pmc_variacao_ES': 'mean',         
    'pmc_indice_Brasil': 'mean',       
    'pmc_indice_sazonal_Brasil': 'mean',
    'pmc_variacao_Brasil': 'mean'       
}).reset_index()

# Exibir o DataFrame resultante
df_origem

In [153]:
df_origem['Vl Liquido Final normalizado'] = np.log1p(df_origem['Vl Liquido Final s/ normalização'])

In [154]:
x_origem = df_origem[['Veiculo','Ano','Mês', 'Taxa Ac. TRI % PIB', 'Origem']] #df apenas com as features desejadas
y_origem = df_origem['Vl Liquido Final normalizado'] #Coluna desejada para a previsão

# Separando os dados em treino (60%), validação (20%) e teste (20%)
X_train_origem, X_temp_origem, y_train_origem, y_temp_origem = train_test_split(x_origem, y_origem, test_size=0.4, random_state=42)

# Dividindo o conjunto temporário (X_temp) em validação e teste
X_val_origem, X_test_origem, y_val_origem, y_test_origem = train_test_split(X_temp_origem, y_temp_origem, test_size=0.5, random_state=42)

In [ ]:
train_data_origem = lgb.Dataset(X_train_origem, label=y_train_origem)
val_data_origem = lgb.Dataset(X_val_origem, label=y_val_origem)

params_origem = {'learning_rate': 0.06, 
          'num_leaves': 126, 
          'feature_fraction': 0.7, 
          'bagging_fraction': 0.9, 
          'bagging_freq': 10, 
          'n_estimators': 790,
          'seed': 42}

callbacks = [lgb.early_stopping(stopping_rounds=100)]
model_origem = lgb.train(params_origem, 
                  train_data_origem, 
                  num_boost_round=1000, 
                  valid_sets=[val_data_origem], 
                  callbacks=callbacks)


y_pred_origem = model_origem.predict(X_test_origem, num_iteration=model_origem.best_iteration)

In [ ]:
# Cálculo das métricas para o modelo
rmse_origem = mean_squared_error(y_test_origem, y_pred_origem, squared=False)
r2_origem = r2_score(y_test_origem, y_pred_origem)
mae_origem = mean_absolute_error(y_test_origem, y_pred_origem)
mape_origem = mean_absolute_percentage_error(y_test_origem, y_pred_origem)

# Exibição das métricas
print(f'Teste - RMSE: {rmse_origem:.4f}')
print(f'Teste - R²: {r2_origem:.4f}')
print(f'Teste - MAE: {mae_origem:.4f}')
print(f'Teste - MAPE: {mape_origem:.4f}')

##### Modelo 3 com otimização dos hiperparâmetros

In [ ]:
# Ignorar avisos
warnings.filterwarnings("ignore")

# definir a função objetivo para a otimização
def objective_origem_opt(trial):
    params_origem_opt = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.07),
        'num_leaves': trial.suggest_int('num_leaves', 31, 130),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.6, 0.8),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.7, 1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'n_estimators': trial.suggest_int('n_estimators', 500, 1000),
        'verbose': -1,  # Desativar logs desnecessários
        'seed': 42  
    }
    
    train_data_origem_opt = lgb.Dataset(X_train_origem, label=y_train_origem)
    val_data_origem_opt = lgb.Dataset(X_val_origem, label=y_val_origem)
    
    model_origem_opt = lgb.train(params_origem_opt, 
                      train_data_origem_opt, 
                      num_boost_round=1000, 
                      valid_sets=[val_data_origem_opt], 
                      callbacks=[lgb.early_stopping(stopping_rounds=100)])  # Sem log_evaluation
    
    y_pred_origem_opt = model_origem_opt.predict(X_val_origem, num_iteration=model_origem_opt.best_iteration)
    
    mape_origem_opt = mean_absolute_percentage_error(y_val_origem, y_pred_origem_opt) * 100
    return mape_origem_opt

# otimização
study_origem_opt = optuna.create_study(direction='minimize')  
study_origem_opt.optimize(objective_origem_opt, n_trials=50)

# mostrar melhores hiperparâmetros
print("Melhores hiperparâmetros encontrados:", study_origem_opt.best_params)

# treinar o modelo com os melhores hiperparâmetros
best_params_origem_opt = study_origem_opt.best_params
best_params_origem_opt['verbose'] = -1  # Desativar logs desnecessários

train_data_origem_opt = lgb.Dataset(X_train_origem, label=y_train_origem)
val_data_origem_opt = lgb.Dataset(X_val_origem, label=y_val_origem)

best_model_origem_opt = lgb.train(best_params_origem_opt, 
                       train_data_origem_opt, 
                       num_boost_round=1000, 
                       valid_sets=[val_data_origem_opt], 
                       callbacks=[lgb.early_stopping(stopping_rounds=100)])

# avaliação final no conjunto de teste
y_pred_test_origem_opt = best_model_origem_opt.predict(X_test_origem, num_iteration=best_model_origem_opt.best_iteration)

In [ ]:
# Cálculo das métricas para o modelo
rmse_origem_opt = mean_squared_error(y_test_origem, y_pred_test_origem_opt, squared=False)
r2_origem_opt = r2_score(y_test_origem, y_pred_test_origem_opt)
mae_origem_opt = mean_absolute_error(y_test_origem, y_pred_test_origem_opt)
mape_origem_opt = mean_absolute_percentage_error(y_test_origem, y_pred_test_origem_opt)

# Exibição das métricas
print('Métricas com a Otimização dos Hiperparâmetros')
print(f'Teste - RMSE: {rmse_origem_opt:.4f}')
print(f'Teste - R²: {r2_origem_opt:.4f}')
print(f'Teste - MAE: {mae_origem_opt:.4f}')
print(f'Teste - MAPE: {mape_origem_opt:.4f}')

In [ ]:
# Plotar valores reais vs previstos para o modelo otimizado
plt.figure(figsize=(12, 6))

# Garantir que os dados de teste estejam alinhados e sem reset de índices desnecessários
plt.plot(y_test_origem.reset_index(drop=True)[:100], label='Valores Reais', color='#6baed6', linestyle='-')
plt.plot(y_pred_test_origem_opt[:100], label='Valores Previstos', color='red', linestyle='--')

# Configurações do gráfico
plt.xlabel('Índice')
plt.ylabel('Valor ( ln(1+x) )')
plt.title('Análise das Predições do Modelo 3: Valores Reais vs. Valores Previstos (Primeiros 100 Dados)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Tamanho do gráfico
plt.figure(figsize=(10, 6))

# Gráfico de dispersão dos valores reais vs previstos
plt.scatter(y_test_origem, y_pred_test_origem_opt, alpha=0.6, edgecolors='k', color='blue', label='Valores reais em comparação com os preditos')

# Adicionar uma linha de referência (linha ideal onde os valores reais são iguais aos previstos)
plt.plot([y_test_origem.min(), y_test_origem.max()], [y_test_origem.min(), y_test_origem.max()], 'r--', lw=2, label='Linha de Posição Ideal')

# Adicionando a legenda
plt.legend(title="Legenda", loc="lower right", fontsize = 10)  # Adiciona a legenda no canto inferior direito

# Rótulos e título
plt.xlabel('Valores Reais ( ln(1+x) )')
plt.ylabel('Valores Preditos ( ln(1+x) )')
plt.title('Diagrama de Dispersão das Predições do Modelo 3 (Veículo e Origem)')

# Adicionar grid
plt.grid(True)

# Exibir gráfico
plt.show()

## 📈 Sistema de Previsões

Nessa seção, será possível realizar as previsões de receita, utilizando o modelo LightGBM. Será necessário fornecer o ano, mês, estimativa da 'Taxa Ac. TRI % PIB', o veículo e, se desejado, a origem ou segmento específico que se deseja prever.

##### ⚠️ Antes de utilizar os sistemas de previsão execute a célula que possui o sistema que deseja utilizar mais uma vez. 

### Modelo 1 - Previsão por Veículo

In [ ]:
# Dicionário com os nomes dos veículos
veiculos_nomes = {
    0: 'CBN VITORIA',
    1: 'GAZETA FM LINHARES',
    2: 'GAZETA FM VITORIA',
    3: 'GAZETA PRODUCOES',
    4: 'INTERNET',
    5: 'LITORAL FM',
    6: 'LITORAL FM NOROESTE',
    7: 'LITORAL FM NORTE',
    8: 'LITORAL FM SUL',
    9: 'PORTAL G1/GE/HOME',
    10: 'PRODUCAO',
    11: 'RADIO MIX VITORIA',
    12: 'TV GAZETA',
    13: 'TV GAZETA NOROESTE',
    14: 'TV GAZETA N0RTE',
    15: 'TV GAZETA SUL',
    16: 'TV NOROESTE PRODUCOES',
    17: 'TV NORTE PRODUCOES',
    18: 'TV SUL PRODUCOES'
}

# Estilo para aumentar o espaço reservado para a descrição
style = {'description_width': '160px'}

# Dropdown para selecionar o veículo
veiculo_dropdown = widgets.Dropdown(
    options=[(v, k) for k, v in veiculos_nomes.items()],
    value=None, 
    description='Veículo:',
    disabled=False
)

# Widgets para o ano e taxas trimestrais
ano_widget_veiculo = widgets.IntText(value=None, description='Ano:')
taxa_ac_q1_veiculo = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 1 (%):', style= style)
taxa_ac_q2_veiculo = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 2 (%):',style= style)
taxa_ac_q3_veiculo = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 3 (%):', style= style)
taxa_ac_q4_veiculo = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 4 (%):', style= style)

# Função para realizar a previsão dos próximos 12 meses com taxas trimestrais diferentes
def fazer_previsao_veiculo(veiculo, ano, taxas):
    # Distribuir as taxas trimestrais ao longo dos meses
    meses = list(range(1, 13))
    taxas_trimestrais = [taxas[0]] * 3 + [taxas[1]] * 3 + [taxas[2]] * 3 + [taxas[3]] * 3
    
    # Criar DataFrame para os 12 meses
    data_para_previsao = pd.DataFrame({
        'Veiculo': [veiculo] * 12,
        'Ano': [ano] * 12,
        'Mês': meses,
        'Taxa Ac. TRI % PIB': taxas_trimestrais
    })
    
    # Fazer previsão com o modelo para os 12 meses
    previsoes_veiculo = model_veiculo.predict(data_para_previsao)
    previsoes_originais_veiculo = np.expm1(previsoes_veiculo)
    
    # Exibir previsões por mês
    print(f'Previsões para {veiculos_nomes[veiculo]} em {ano}:')
    for mes, previsao_original in zip(meses, previsoes_originais_veiculo):
        print(f'Mês {mes}: {previsao_original:.2f}')

     # Gerar gráfico de linhas com as previsões
    plt.figure(figsize=(10, 6))
    plt.plot(meses, previsoes_originais_veiculo, marker='o', linestyle='-', color='b', label='Previsão')
    plt.title(f'Previsões Mensais para {veiculos_nomes[veiculo]} - ({ano})')
    plt.xlabel('Mês')
    plt.ylabel('Previsão de Receita (R$)')
    plt.xticks(meses)
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Mostrar resultados anteriores separadamente para cada mês
    for mes in meses:
        resultados_anteriores = df_veiculo[['Mês', 'Ano', 'Vl Liquido Final s/ normalização']].loc[
            (df_veiculo['Veiculo'] == veiculo) & (df_veiculo['Mês'] == mes)
        ]
        
        print(f'\nReceitas anteriores - Mês {mes}:')
        display(resultados_anteriores)

# Botão para acionar a previsão
botao_previsao_veiculo = widgets.Button(description="Fazer Previsão")

# Função do botão
def on_button_click(b):
    taxas_veiculo = [taxa_ac_q1_veiculo.value, taxa_ac_q2_veiculo.value, taxa_ac_q3_veiculo.value, taxa_ac_q4_veiculo.value]
    fazer_previsao_veiculo(veiculo_dropdown.value, ano_widget_veiculo.value, taxas_veiculo)

botao_previsao_veiculo.on_click(on_button_click)

# Exibir os widgets e o botão
display(veiculo_dropdown, ano_widget_veiculo, taxa_ac_q1_veiculo, taxa_ac_q2_veiculo, taxa_ac_q3_veiculo, taxa_ac_q4_veiculo, botao_previsao_veiculo)

### Modelo 2 - Previsão por Veículo e Segmento

In [ ]:
# Dicionários com os nomes dos segmentos
segmentos_nomes = {
    0: 'AGROPECUARIA',
    1: 'ALIMENTOS',
    2: 'BEBIDAS',
    3: 'BENS INDUSTRIAIS MAT. PRIMA',
    4: 'BRINQUEDOS E DIVERSOES',
    5: 'COMERCIO',
    6: 'CONDOMINIOS',
    7: 'EDUCACAO/MEIOS DE COMUNICACAO',
    8: 'EQUIP/MAT/ESCRIT/LOJA/ESCOLA',
    9: 'FOTO / OTICA / CINE / SOM',
    10: 'INDUSTRIA DA CONSTRUCAO',
    11: 'INTERNET',
    12: 'LIMPEZA / HIGIENE DOMESTICA',
    13: 'MEIOS DE TRANSPORTES E AFINS',
    14: 'MERCADO FINANCEIRO',
    15: 'MOVEIS E DECORACOES',
    16: 'OUTRAS RECEITAS',
    17: 'OUTROS',
    18: 'PERFUMARIA / FARMACIA',
    19: 'SERVICOS',
    20: 'SERVICOS DE TELECOMUNICACOES',
    21: 'TEXTIL / VESTUARIO',
    22: 'UTILIDADES DOMESTICAS'
}

# Estilo para ajustar a largura da descrição dos widgets
style = {'description_width': '160px'}

# Dropdowns para selecionar o veículo e o segmento
veiculo_dropdown = widgets.Dropdown(
    options=[(v, k) for k, v in veiculos_nomes.items()],
    value=None,  # Default para 'TV GAZETA'
    description='Veículo:',
    disabled=False,
    style=style
)

segmento_dropdown = widgets.Dropdown(
    options=[(v, k) for k, v in segmentos_nomes.items()],
    value=None,  # Default para 'AGROPECUARIA'
    description='Segmento:',
    disabled=False,
    style=style
)

# Widgets para o ano e as taxas trimestrais
ano_widget_segmento = widgets.IntText(value=None, description='Ano:', style=style)
taxa_ac_q1_segmento = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 1 (%):', style=style)
taxa_ac_q2_segmento = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 2 (%):', style=style)
taxa_ac_q3_segmento = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 3 (%):', style=style)
taxa_ac_q4_segmento = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 4 (%):', style=style)

# Função para realizar a previsão
def fazer_previsao_segmento(veiculo, segmento, ano, taxas):
    meses = list(range(1, 13))
    taxas_trimestrais = [taxas[0]] * 3 + [taxas[1]] * 3 + [taxas[2]] * 3 + [taxas[3]] * 3
    
    # Criar DataFrame para os 12 meses
    data_para_previsao = pd.DataFrame({
        'Veiculo': [veiculo] * 12,
        'Ano': [ano] * 12,
        'Mês': meses,
        'Segmento': [segmento] * 12,
        'Taxa Ac. TRI % PIB': taxas_trimestrais
    })
    
    # Fazer previsão com o modelo para os 12 meses
    previsoes_segmento = model_segmento.predict(data_para_previsao)
    previsoes_originais_segmento = np.expm1(previsoes_segmento)
    
    # Exibir previsões por mês
    print(f'Previsões para {veiculos_nomes[veiculo]} no segmento {segmentos_nomes[segmento]} em {ano}:')
    for mes, previsao_original in zip(meses, previsoes_originais_segmento):
        print(f'Mês {mes}: {previsao_original:.2f}')

         # Gerar gráfico de linhas com as previsões
    plt.figure(figsize=(10, 6))
    plt.plot(meses, previsoes_originais_segmento, marker='o', linestyle='-', color='b', label='Previsão')
    plt.title(f'Previsões Mensais para {veiculos_nomes[veiculo]} - Segmento {segmentos_nomes[segmento]} ({ano})')
    plt.xlabel('Mês')
    plt.ylabel('Previsão de Receita (R$)')
    plt.xticks(meses)
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Mostrar receitas anteriores para cada mês
    for mes in meses:
        resultados_anteriores = df_segmento[['Mês', 'Ano', 'Vl Liquido Final s/ normalização']].loc[
            (df_segmento['Veiculo'] == veiculo) & 
            (df_segmento['Mês'] == mes) & 
            (df_segmento['Segmento'] == segmento)
        ]
        
        print(f'\nReceitas anteriores - Mês {mes}:')
        display(resultados_anteriores)

# Botão para acionar a previsão
botao_previsao_segmento = widgets.Button(description="Fazer Previsão")

# Função do botão
def on_button_click(b):
    taxas_segmento = [taxa_ac_q1_segmento.value, taxa_ac_q2_segmento.value, taxa_ac_q3_segmento.value, taxa_ac_q4_segmento.value]
    fazer_previsao_segmento(veiculo_dropdown.value, segmento_dropdown.value, ano_widget_segmento.value, taxas_segmento)

botao_previsao_segmento.on_click(on_button_click)

# Exibir os widgets e o botão
display(veiculo_dropdown, segmento_dropdown, ano_widget_segmento, taxa_ac_q1_segmento, taxa_ac_q2_segmento, taxa_ac_q3_segmento, taxa_ac_q4_segmento, botao_previsao_segmento)

### Modelo 3 - Previsão por Veículo e Origem

In [ ]:
# Dicionários com os nomes das origens
origens_nomes = {
    0: 'CH - CONTATO - CACHOEIRO',
    1: 'CO - CONTATO - COLATINA',
    2: 'LI - CONTATO - LINHARES',
    3: 'MP - MÍDIA PROGRAMÁTICA',
    4: 'RN - MERCADO NACIONAL',
    5: 'VT - CONTATO - VITÓRIA'
}

# Estilo para ajustar a largura da descrição dos widgets
style = {'description_width': '160px'}

# Dropdowns para selecionar o veículo e o segmento
veiculo_dropdown = widgets.Dropdown(
    options=[(v, k) for k, v in veiculos_nomes.items()],
    value=None, 
    description='Veículo:',
    disabled=False,
    style=style
)

origem_dropdown = widgets.Dropdown(
    options=[(v, k) for k, v in origens_nomes.items()],
    value=None,  # Default 
    description='Origem:',
    disabled=False,
    style=style
)

# Widgets para o ano e as taxas trimestrais
ano_widget_origem = widgets.IntText(value=None, description='Ano:', style=style)
taxa_ac_q1_origem = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 1 (%):', style=style)
taxa_ac_q2_origem = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 2 (%):', style=style)
taxa_ac_q3_origem = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 3 (%):', style=style)
taxa_ac_q4_origem = widgets.FloatText(value=None, description='Taxa Ac. do PIB - TRI 4 (%):', style=style)

# Função para realizar a previsão
def fazer_previsao_origem(veiculo, origem, ano, taxas):
    meses = list(range(1, 13))
    taxas_trimestrais = [taxas[0]] * 3 + [taxas[1]] * 3 + [taxas[2]] * 3 + [taxas[3]] * 3
    
    # Criar DataFrame para os 12 meses
    data_para_previsao = pd.DataFrame({
        'Veiculo': [veiculo] * 12,
        'Ano': [ano] * 12,
        'Mês': meses,
        'Origem': [origem] * 12,
        'Taxa Ac. TRI % PIB': taxas_trimestrais
    })
    
    # Fazer previsão com o modelo para os 12 meses
    previsoes_origem = model_origem.predict(data_para_previsao)
    previsoes_originais_origem = np.expm1(previsoes_origem)
    
    # Exibir previsões por mês
    print(f'Previsões para {veiculos_nomes[veiculo]} na origem {origens_nomes[origem]} em {ano}:')
    for mes, previsao_original in zip(meses, previsoes_originais_origem):
        print(f'Mês {mes}: {previsao_original:.2f}')

     # Gerar gráfico de linhas com as previsões
    plt.figure(figsize=(10, 6))
    plt.plot(meses, previsoes_originais_origem, marker='o', linestyle='-', color='b', label='Previsão')
    plt.title(f'Previsões Mensais para {veiculos_nomes[veiculo]} - Origem {origens_nomes[origem]} ({ano})')
    plt.xlabel('Mês')
    plt.ylabel('Previsão de Receita (R$)')
    plt.xticks(meses)
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Mostrar receitas anteriores para cada mês
    for mes in meses:
        resultados_anteriores = df_origem[['Mês', 'Ano', 'Vl Liquido Final s/ normalização']].loc[
            (df_origem['Veiculo'] == veiculo) & 
            (df_origem['Mês'] == mes) & 
            (df_origem['Origem'] == origem)
        ]
        
        print(f'\nReceitas anteriores - Mês {mes}:')
        display(resultados_anteriores)

# Botão para acionar a previsão
botao_previsao_origem = widgets.Button(description="Fazer Previsão")

# Função do botão
def on_button_click(b):
    taxas_origem = [taxa_ac_q1_origem.value, taxa_ac_q2_origem.value, taxa_ac_q3_origem.value, taxa_ac_q4_origem.value]
    fazer_previsao_origem(veiculo_dropdown.value, origem_dropdown.value, ano_widget_origem.value, taxas_origem)

botao_previsao_origem.on_click(on_button_click)

# Exibir os widgets e o botão
display(veiculo_dropdown, origem_dropdown, ano_widget_origem, taxa_ac_q1_origem, taxa_ac_q2_origem, taxa_ac_q3_origem, taxa_ac_q4_origem, botao_previsao_origem)

## 📃 Referências

PYTHON. Documentação Python. Wiki Python Brasil. Disponível em: https://wiki.python.org.br/DocumentacaoPython. Acesso em: 11 set. 2024.

MATPLOTLIB. matplotlib.pyplot API. Versão 3.5.3. Disponível em: https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html. Acesso em: 11 set. 2024.

SCIPY. SciPy Documentation. Disponível em: https://docs.scipy.org/doc/scipy/. Acesso em: 11 set. 2024.

PANDAS. Pandas Documentation. Disponível em: https://pandas.pydata.org/docs/index.html#. Acesso em: 11 set. 2024.

NUMPY. NumPy Documentation. Disponível em: https://numpy.org/devdocs/. Acesso em: 11 set. 2024.

SCIKIT-LEARN. Scikit-learn Documentation. Versão 0.21. Disponível em: https://scikit-learn.org/0.21/documentation.html. Acesso em: 11 set. 2024.

SCIPY. SciPy Documentation. Disponível em: https://docs.scipy.org/doc/. Acesso em: 11 set. 2024.

SEABORN. Seaborn Documentation. Disponível em: https://seaborn.pydata.org/. Acesso em: 11 set. 2024.